In [1]:
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from google.auth.transport.requests import Request
import os.path
import json
import pickle
import requests
import asyncio
from tenacity import retry, wait_exponential, stop_after_attempt, retry_if_exception_type

def oauth():
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 
              'https://www.googleapis.com/auth/script.projects', 'https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/drive.file'
              ]
    creds_filename = 'test-daebong-service-account.json'  # 서비스 계정 파일 경로를 지정합니다.

    # 서비스 계정 파일을 사용하여 인증 정보를 로드합니다.
    creds = service_account.Credentials.from_service_account_file(creds_filename, scopes=SCOPES)
    return creds

def oauthByWeb():
    # 필요한 스코프를 지정합니다.
    SCOPES = [
        'https://www.googleapis.com/auth/spreadsheets',
        'https://www.googleapis.com/auth/script.projects',
        'https://www.googleapis.com/auth/drive',
        'https://www.googleapis.com/auth/drive.file'
    ]
    creds = None
    # 'token.json' 파일이 존재하면, 저장된 인증 정보를 불러옵니다.
    if os.path.exists('token.json'):
        with open('token.json', 'rb') as token:
            creds = pickle.load(token)

    # 저장된 인증 정보가 없거나, 유효하지 않은 경우 새로운 인증을 진행합니다.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            creds_filename = 'oauth-new-daebong.json'
            flow = InstalledAppFlow.from_client_secrets_file(creds_filename, SCOPES)
            creds = flow.run_local_server(port=8080)
        # 새로운 인증 정보를 'token.json'에 저장합니다.
        with open('token.json', 'wb') as token:
            pickle.dump(creds, token)

    return creds



In [2]:
@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5), retry=retry_if_exception_type(HttpError))
def importrangeAllowAccessAutomation(get_spreadsheet_id, give_spreadsheet_id, creds):
    url = f'https://docs.google.com/spreadsheets/d/{get_spreadsheet_id}/externaldata/addimportrangepermissions?donorDocId={give_spreadsheet_id}&includes_info_params=true&cros_files=false'
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {creds.token}',
    }
    # 데이터를 JSON 형태로 변환하고, POST 요청을 보냅니다.
    response = requests.post(url, headers=headers)
    # 응답 확인
    if 'application/json' in response.headers.get('Content-Type', ''):
        try:
            data = response.json()
            print(data)
        except ValueError:
            print("JSON 디코딩 실패", response.text)
    else:
        print("응답이 JSON 형식이 아닙니다:", response.text)

    

In [3]:
@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5), retry=retry_if_exception_type(HttpError))
async def batchWriteRequest(service, spreadsheet_id, all_requests):
    # 요청들을 하나의 리스트로 병합
    # all_requests = []
    # for req in requests:
    #     all_requests.append(req)
    
    # 요청 바디 작성
    body = {
        'requests': all_requests
    }

    # batchUpdate 호출
    response = service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body=body
    ).execute()
    
    return response


In [4]:
def col_to_index(col):
    index = 0
    for c in col:
        index = index * 26 + (ord(c) - ord('A') + 1)
    return index - 1

In [5]:
def convert_range_to_indices(sheet_id, cell_range):
    """
    주어진 셀 범위를 rowIndex와 columnIndex로 변환합니다.
    
    :param sheet_id: 변환할 시트의 ID
    :param cell_range: 변환할 셀 범위 (예: "A2:B")
    :return: 변환된 범위 딕셔너리
    """
    import re
    
    # A1 표기법을 정규 표현식으로 분리
    match = re.match(r"([A-Z]+)([0-9]+):([A-Z]+)([0-9]+)", cell_range)
    if not match:
        raise ValueError("셀 범위 형식이 올바르지 않습니다. 예: 'A2:B10'")
    
    start_col, start_row, end_col, end_row = match.groups()
    
    start_row_index = int(start_row) - 1
    end_row_index = int(end_row)
    start_col_index = col_to_index(start_col)
    end_col_index = col_to_index(end_col) + 1
    
    return {
        'sheetId': sheet_id,
        'startRowIndex': start_row_index,
        'endRowIndex':end_row_index,
        'startColumnIndex': start_col_index,
        'endColumnIndex': end_col_index
    }

In [6]:
def apply_protect_data(sheet_order_id, range_name):
    # 스프레드시트의 메타데이터에서 sheet_id 가져오기
    range_value = convert_range_to_indices(sheet_order_id, range_name);
    user_emails = ["daebong10x@gmail.com", "test-daebong@newdaebong.iam.gserviceaccount.com"]

        # 보호된 범위 설정
    request = {
        'addProtectedRange': {
            'protectedRange': {
                'range': range_value,
                'description': '수정을 원한다면 문의주세요',
                'editors': {
                    'users': user_emails,  # 이 셀을 편집할 수 있는 사용자
                }
            }
        }
    }

    return request

In [7]:
# 특정 셀에 수식을 적용하는 함수
def apply_formula_to_text(text, cell_range, sheet_order_id):
    range_name = f"{cell_range}"  # 지정된 시트의 특정 셀 범위    
    
    range_value = convert_range_to_indices(sheet_order_id, range_name);

    request = {
        "updateCells": {
            "rows": [
                {
                    "values": [
                        {"userEnteredValue": {"stringValue": text}}
                    ]
                }
            ],
            "fields": "userEnteredValue,userEnteredFormat.textFormat",
            "range": range_value,
        }
    }
    return request

In [8]:
#sheet_id 한 번에 호출해서 타이틀마다 변환해야할 것들을 가져와야한다. 
def apply_formula_to_function(formula, cell_range, sheet_order_id):
    range_name = f"{cell_range}"  # 지정된 시트의 특정 셀 범위    
    # spreadsheet = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
    range_value = convert_range_to_indices(sheet_order_id, range_name);

    request = {
        "updateCells": {
            "rows": [
                {
                    "values": [
                        {"userEnteredValue": {
                            "formulaValue": formula
                        }}
                    ]
                }
            ],
            "fields": "userEnteredValue,userEnteredFormat.textFormat",
            "range": range_value,
        }
    }
    return request

In [9]:
def apply_dropdown_validation(dropdown_range, cell_range, sheet_order_id):
    range_name = f"{cell_range}"  # 지정된 시트의 특정 셀 범위    
    # spreadsheet = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
    range_value = convert_range_to_indices(sheet_order_id, range_name);
    
    # 드롭다운 유효성 검사 설정
    request = {
                'repeatCell': {
                    'range': range_value,
                    'cell': {
                        'dataValidation': {
                            'condition': {
                                'type': 'ONE_OF_RANGE',
                                'values': [
                                    {
                                        'userEnteredValue': f"={dropdown_range}"
                                    }
                                ]
                            },
                            'showCustomUi': True, # 드롭다운 화살표 표시
                            'strict': False # 경고 표시
                        }
                    },
                    'fields': 'dataValidation'
                }
    }
    
    return request

In [10]:
def apply_rename_request(sheet_id, title):
    request = {
                "updateSheetProperties": {
                    "properties": {
                        "sheetId": sheet_id,
                        "title": title,
                    },
                    "fields": "title"
                }
    }
    return request

In [11]:
def apply_index_request(sheet_id, index):
    request =  {
                "updateSheetProperties": {
                    "properties": {
                        "sheetId": sheet_id,
                        "index": index
                    },
                    "fields": "index"
                }
    }
    return request

In [12]:
#sheet_id 한 번에 호출해서 타이틀마다 변환해야할 것들을 가져와야한다. 
def apply_formula_product_name_to_function_in_everyrow(sheet_order_id):
    requests = []
    
    for i in range(12, 900):  # 11행부터 1010행까지
        formula = f'=IF($A$8="매핑모드",IFERROR(INDEX(\'상품목록\'!C:C, MATCH(1, (E{i}=\'상품목록\'!A:A) * (F{i}=\'상품목록\'!B:B), 0)), ""),    IF($A$8="일반모드",  D{i}, "" ))'
        requests.append(
            {
            "updateCells": {
                "range": {
                    "sheetId": sheet_order_id,
                    "startRowIndex": i - 1,  # 0-based index
                    "endRowIndex": i,
                    "startColumnIndex":6,  # Column G (0-based index)
                    "endColumnIndex": 7
                },
                "rows": [
                    {
                        "values": [
                            {
                                "userEnteredValue": {
                                    "formulaValue": formula
                                }
                            }
                        ]
                    }
                ],
                "fields": "userEnteredValue"
            }
        }
        )
    return requests

In [13]:
#sheet_id 한 번에 호출해서 타이틀마다 변환해야할 것들을 가져와야한다. 
def apply_formula_product_name_to_function_in_everyrow_mapping_mode(sheet_order_id):
    requests = []
    
    for i in range(12, 900):  # 11행부터 1010행까지
        formula = f'=IFERROR(INDEX(\'상품목록\'!C:C, MATCH(1, (E{i}=\'상품목록\'!A:A) * (F{i}=\'상품목록\'!B:B), 0)), "")'
        requests.append(
            {
            "updateCells": {
                "range": {
                    "sheetId": sheet_order_id,
                    "startRowIndex": i - 1,  # 0-based index
                    "endRowIndex": i,
                    "startColumnIndex":6,  # Column G (0-based index)
                    "endColumnIndex": 7
                },
                "rows": [
                    {
                        "values": [
                            {
                                "userEnteredValue": {
                                    "formulaValue": formula
                                }
                            }
                        ]
                    }
                ],
                "fields": "userEnteredValue"
            }
        }
        )
    return requests

In [14]:
#sheet_id 한 번에 호출해서 타이틀마다 변환해야할 것들을 가져와야한다. 
def apply_formula_mapping_check_to_function_in_everyrow(sheet_order_id):
    requests = []
    
    for i in range(12, 900):  # 11행부터 1010행까지
        formula = f'=IF($A$8="매핑모드", IF(E{i}<>"", IF(G{i}<>"", "정상매핑", "매핑오류"), ""), "")'
        requests.append({
            "updateCells": {
                "range": {
                    "sheetId": sheet_order_id,
                    "startRowIndex": i - 1,  # 0-based index
                    "endRowIndex": i,
                    "startColumnIndex":7,  # Column H (0-based index)
                    "endColumnIndex": 8
                },
                "rows": [
                    {
                        "values": [
                            {
                                "userEnteredValue": {
                                    "formulaValue": formula
                                }
                            }
                        ]
                    }
                ],
                "fields": "userEnteredValue"
            }
        })
    return requests

In [15]:
#sheet_id 한 번에 호출해서 타이틀마다 변환해야할 것들을 가져와야한다. 
def apply_formula_product_price_to_function_in_everyrow(sheet_order_id):
    requests = []
    
    for i in range(12, 900):  # 11행부터 1010행까지
        formula = f'=IFERROR(IF(INDEX(\'상품목록\'!L:L, MATCH(\'발주서\'!G{i}, \'상품목록\'!K:K, 0)) * \'발주서\'!L{i} = 0, "", INDEX(\'상품목록\'!L:L, MATCH(\'발주서\'!G{i}, \'상품목록\'!K:K, 0)) * \'발주서\'!L{i}))'
        requests.append({
            "updateCells": {
                "range": {
                    "sheetId": sheet_order_id,
                    "startRowIndex": i - 1,  # 0-based index
                    "endRowIndex": i,
                    "startColumnIndex":9,  # Column J (0-based index)
                    "endColumnIndex": 10
                },
                "rows": [
                    {
                        "values": [
                            {
                                "userEnteredValue": {
                                    "formulaValue": formula
                                }
                            }
                        ]
                    }
                ],
                "fields": "userEnteredValue"
            }
        })
    return requests

In [16]:
#sheet_id 한 번에 호출해서 타이틀마다 변환해야할 것들을 가져와야한다. 
def apply_formula_order_status_to_function_in_everyrow(sheet_order_id):
    requests = []
    
    for i in range(12, 900):  # 11행부터 1010행까지
        formula = f'=IF($A$8="매핑모드",IF(ISNUMBER(SEARCH("정상매핑", H{i})),IF(AND(A{i}<>"", B{i}<>"", C{i}<>"", J{i}<>""), "정상입력", "필수입력사항 입력필요"),IF(H{i}="매핑오류", "매핑 확인필요", IF(OR(A{i}<>"", B{i}<>"", C{i}<>"", J{i}<>""), "매핑 확인필요", ""))),IF($A$8="일반모드",IF(AND(A{i}="", B{i}="", C{i}="", D{i}="", J{i}=""), "", IF(OR(A{i}="", B{i}="", C{i}="", D{i}="", J{i}=""), "필수입력사항 입력필요", "정상입력")),""))'
        requests.append({
            "updateCells": {
                "range": {
                    "sheetId": sheet_order_id,
                    "startRowIndex": i - 1,  # 0-based index
                    "endRowIndex": i,
                    "startColumnIndex":10,  # Column K (0-based index)
                    "endColumnIndex": 11
                },
                "rows": [
                    {
                        "values": [
                            {
                                "userEnteredValue": {
                                    "formulaValue": formula
                                }
                            }
                        ]
                    }
                ],
                "fields": "userEnteredValue"
            }
        })
    return requests

In [17]:
#sheet_id 한 번에 호출해서 타이틀마다 변환해야할 것들을 가져와야한다. 
def apply_formula_product_number_to_function_in_everyrow(sheet_order_id):

    requests = []

    for i in range(12, 900):  # 11행부터 1010행까지
        formula = f'=IFERROR(IF(LEN(G{i})=0,NA(), 1))'
        requests.append({
            "updateCells": {
                "range": {
                    "sheetId": sheet_order_id,
                    "startRowIndex": i - 1,  # 0-based index
                    "endRowIndex": i,
                    "startColumnIndex": 11,  # Column J (0-based index)
                    "endColumnIndex": 12
                },
                "rows": [
                    {
                        "values": [
                            {
                                "userEnteredValue": {
                                    "formulaValue": formula
                                }
                            }
                        ]
                    }
                ],
                "fields": "userEnteredValue"
            }
        })
    return requests

In [18]:
#sheet_id 한 번에 호출해서 타이틀마다 변환해야할 것들을 가져와야한다. 
def apply_formula_code_number_to_function_in_everyrow(sheet_order_id, new_daebong_spreadsheet_title):

    requests = []

    for i in range(12, 900):  # 11행부터 1010행까지
        formula = f'=IF(LEN(A{i})=0,"", {new_daebong_spreadsheet_title})'
        requests.append({
            "updateCells": {
                "range": {
                    "sheetId": sheet_order_id,
                    "startRowIndex": i - 1,  # 0-based index
                    "endRowIndex": i,
                    "startColumnIndex": 19,  # Column T (0-based index)
                    "endColumnIndex": 20
                },
                "rows": [
                    {
                        "values": [
                            {
                                "userEnteredValue": {
                                    "formulaValue": formula
                                }
                            }
                        ]
                    }
                ],
                "fields": "userEnteredValue"
            }
        })
    return requests

In [19]:
#sheet_id 한 번에 호출해서 타이틀마다 변환해야할 것들을 가져와야한다. 
def apply_formula_to_function_in_everyrow(sheet_order_id, new_daebong_spreadsheet_title):

    requests = []

    for i in range(12, 900):  # 11행부터 1010행까지
        formula = f'=IF(LEN(A{i})=0,"", {new_daebong_spreadsheet_title})'
        requests.append({
            "updateCells": {
                "range": {
                    "sheetId": sheet_order_id,
                    "startRowIndex": i - 1,  # 0-based index
                    "endRowIndex": i,
                    "startColumnIndex": 19,  # Column J (0-based index)
                    "endColumnIndex": 20
                },
                "rows": [
                    {
                        "values": [
                            {
                                "userEnteredValue": {
                                    "formulaValue": formula
                                }
                            }
                        ]
                    }
                ],
                "fields": "userEnteredValue"
            }
        })
    return requests

In [20]:
#sheet_id 한 번에 호출해서 타이틀마다 변환해야할 것들을 가져와야한다. 
def apply_formula_delivery_company_to_function_in_everyrow(sheet_order_id):
    requests = []
    
    for i in range(12, 900):  # 11행부터 1010행까지
        formula = f'=IFERROR(VLOOKUP(G{i}, \'상품목록\'!K$6:M, 3, FALSE), "")'
        requests.append({
            "updateCells": {
                "range": {
                    "sheetId": sheet_order_id,
                    "startRowIndex": i - 1,  # 0-based index
                    "endRowIndex": i,
                    "startColumnIndex":9,  # Column K (0-based index)
                    "endColumnIndex": 10
                },
                "rows": [
                    {
                        "values": [
                            {
                                "userEnteredValue": {
                                    "formulaValue": formula
                                }
                            }
                        ]
                    }
                ],
                "fields": "userEnteredValue"
            }
        })
    return requests

In [21]:
#sheet_id 한 번에 호출해서 타이틀마다 변환해야할 것들을 가져와야한다. 
def apply_formula_suppy_price_to_function_in_everyrow(sheet_order_id):
    requests = []
    
    for i in range(12, 900):  # 11행부터 1010행까지
        formula = f'=IFERROR(IF(INDEX(\'상품목록\'!L:L, MATCH(\'발주서\'!G{i}, \'상품목록\'!K:K, 0))*\'발주서\'!L{i}=0, "", INDEX(\'상품목록\'!L:L, MATCH(\'발주서\'!G{i}, \'상품목록\'!K:K, 0))*\'발주서\'!L{i}))'
        requests.append({
            "updateCells": {
                "range": {
                    "sheetId": sheet_order_id,
                    "startRowIndex": i - 1,  # 0-based index
                    "endRowIndex": i,
                    "startColumnIndex":52,  # Column J (0-based index)
                    "endColumnIndex": 53
                },
                "rows": [
                    {
                        "values": [
                            {
                                "userEnteredValue": {
                                    "formulaValue": formula
                                }
                            }
                        ]
                    }
                ],
                "fields": "userEnteredValue"
            }
        })
    return requests

In [22]:
@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5), retry=retry_if_exception_type(HttpError))
def find_last_row_in_spreadsheet(service, spreadsheet_id, column):
    """
    지정된 열에서 데이터가 있는 마지막 행의 인덱스를 반환합니다.
    """
    range_name = f'{column}:{column}'  # 예: 'D:D'
    result = service.spreadsheets().values().get(
        spreadsheetId=spreadsheet_id,
        range=range_name
    ).execute()
    values = result.get('values', [])

    if not values:
        return 1  # 데이터가 없으면 첫 번째 행 반환
    else:
        return len(values) + 1  # 데이터가 있는 마지막 행의 다음 행 반환



def append_to_sheet_in_a_row(service, many_ids, sheet_order_id):
    last_row = find_last_row_in_spreadsheet(service, new_spreadsheet_id, 'D')
    # D열의 마지막 행 찾기
    # last_row_D = find_last_row(service, spreadsheet_id, 'D')
    results = []
    count = 0

    for order_sheet_title, new_spreadsheet_id in many_ids:
        this_row = last_row + count
        count += 1

        range_name_D = f'시트1!D{this_row}'
        request_D = apply_formula_to_text(order_sheet_title, range_name_D, sheet_order_id)

        # E열의 마지막 행 찾
        range_name_E = f'시트1!E{this_row}'
        request_E = apply_formula_to_text(new_spreadsheet_id, range_name_E, sheet_order_id)

        spreadsheet_url = f"https://docs.google.com/spreadsheets/d/{new_spreadsheet_id}"

        range_name_F = f'시트1!F{this_row}'
        request_F = apply_formula_to_text(spreadsheet_url, range_name_F, sheet_order_id)

        results.extend([request_D, request_E, request_F])

    return results 

    # return [request_D, request_E. request_F]



            


In [23]:
import requests

@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5), retry=retry_if_exception_type(HttpError))
def importrangeAllowAccessAutomation(get_spreadsheet_id, give_spreadsheet_id, creds):
    url = f'https://docs.google.com/spreadsheets/d/{get_spreadsheet_id}/externaldata/addimportrangepermissions?donorDocId={give_spreadsheet_id}&token={creds.token}&includes_info_params=true&cros_files=false'
    # https://docs.google.com/spreadsheets/d/1YwT7BaFpOKB2Ng3MViX7UGuF9hZJUJRBFJN3eXn4gHw/externaldata/addimportrangepermissions?donorDocId=1Fg76vk3lucUwwvmcoQJVb4KiMfjJSNx2VEFSdhwYFy4&token=AC4w5VjkIUgDXtWZH968yYaAfiyhZJcApg%3A1718330258244&includes_info_params=true&cros_files=false

    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {creds.token}',
    }
    # 데이터를 JSON 형태로 변환하고, POST 요청을 보냅니다.
    response = requests.post(url, headers=headers)
    # 응답 확인
    if 'application/json' in response.headers.get('Content-Type', ''):
        try:
            # 응답 상태 코드 확인
            if response.status_code == 200:
                # JSON 응답 파싱
                data = response.json()
                # 파싱된 데이터 출력
                print(data)
                print(response);
            # data = response.json()
            # print(data)
        except ValueError:
            print("JSON 디코딩 실패", response.text)
    else:
        print("응답이 JSON 형식이 아닙니다:", response.text)

    

In [24]:
@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5), retry=retry_if_exception_type(HttpError))
async def copy_sheet_to_another_spreadsheet(service, source_spreadsheet_id, source_sheet_id, destination_spreadsheet_id):


    request_body = {
        'destinationSpreadsheetId': destination_spreadsheet_id
    }
    
    response = service.spreadsheets().sheets().copyTo(
        spreadsheetId=source_spreadsheet_id,
        sheetId=source_sheet_id,
        body=request_body
    ).execute()
    
    return response['sheetId']

@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5), retry=retry_if_exception_type(HttpError))
async def main_copy_sheet(service_sheets, source_spreadsheet_id, destination_spreadsheet_id):

    #특정시트에 복사하기 
    spreadsheet = service_sheets.spreadsheets().get(spreadsheetId=source_spreadsheet_id).execute()
    sheets = spreadsheet.get('sheets', '')
    sheet_order_id_발주서 = None
    sheet_order_id_매핑모드 = None
    sheet_order_id_누적발주 = None
    sheet_order_id_입금내역 = None
    sheet_order_id_상품목록 = None
    sheet_order_id_예판상품 = None
    
    for sheet in sheets:
        if sheet.get('properties', {}).get('title', '') == "발주서":
            sheet_order_id_발주서 = sheet.get('properties', {}).get('sheetId', '')
        if sheet.get('properties', {}).get('title', '') == "발주서매핑모드":
            sheet_order_id_매핑모드 = sheet.get('properties', {}).get('sheetId', '')
        if sheet.get('properties', {}).get('title', '') == "누적발주":
            sheet_order_id_누적발주 = sheet.get('properties', {}).get('sheetId', '')
        if sheet.get('properties', {}).get('title', '') == "입금내역":
            sheet_order_id_입금내역 = sheet.get('properties', {}).get('sheetId', '')
        if sheet.get('properties', {}).get('title', '') == "상품목록":
            sheet_order_id_상품목록 = sheet.get('properties', {}).get('sheetId', '')
        if sheet.get('properties', {}).get('title', '') == "예판상품":
            sheet_order_id_예판상품 = sheet.get('properties', {}).get('sheetId', '')

    new_sheet_order_id_발주서 = await copy_sheet_to_another_spreadsheet(service_sheets, source_spreadsheet_id, sheet_order_id_발주서, destination_spreadsheet_id)
    new_sheet_order_id_매핑모드 = await copy_sheet_to_another_spreadsheet(service_sheets, source_spreadsheet_id, sheet_order_id_매핑모드, destination_spreadsheet_id)
    new_sheet_order_id_누적발주 = await copy_sheet_to_another_spreadsheet(service_sheets, source_spreadsheet_id, sheet_order_id_누적발주, destination_spreadsheet_id)
    new_sheet_order_id_입금내역 = await copy_sheet_to_another_spreadsheet(service_sheets, source_spreadsheet_id, sheet_order_id_입금내역, destination_spreadsheet_id)
    new_sheet_order_id_상품목록 = await copy_sheet_to_another_spreadsheet(service_sheets, source_spreadsheet_id, sheet_order_id_상품목록, destination_spreadsheet_id)
    new_sheet_order_id_예판상품 = await copy_sheet_to_another_spreadsheet(service_sheets, source_spreadsheet_id, sheet_order_id_예판상품, destination_spreadsheet_id)


    return [new_sheet_order_id_발주서,new_sheet_order_id_누적발주,new_sheet_order_id_입금내역,new_sheet_order_id_상품목록, new_sheet_order_id_매핑모드, new_sheet_order_id_예판상품]

@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5), retry=retry_if_exception_type(HttpError))
async def delete_sheet_by_title(service_sheets, destination_spreadsheet_id):
    spreadsheet = service_sheets.spreadsheets().get(spreadsheetId=destination_spreadsheet_id).execute()
    sheets = spreadsheet.get('sheets', '')
    sheet_order_id_발주서 = None
    sheet_order_id_누적발주 = None
    sheet_order_id_입금내역 = None
    sheet_order_id_상품목록 = None
    
    for sheet in sheets:
        if sheet.get('properties', {}).get('title', '') == "발주서" or sheet.get('properties', {}).get('title', '') == "발주서의 사본" :
            sheet_order_id_발주서 = sheet.get('properties', {}).get('sheetId', '')
        if sheet.get('properties', {}).get('title', '') == "누적발주" or sheet.get('properties', {}).get('title', '') == "누적발주의 사본":
            sheet_order_id_누적발주 = sheet.get('properties', {}).get('sheetId', '')
        if sheet.get('properties', {}).get('title', '') == "입금내역" or sheet.get('properties', {}).get('title', '') == "입금내역의 사본":
            sheet_order_id_입금내역 = sheet.get('properties', {}).get('sheetId', '')
        if sheet.get('properties', {}).get('title', '') == "상품목록" or sheet.get('properties', {}).get('title', '') == "상품목록의 사본":
            sheet_order_id_상품목록 = sheet.get('properties', {}).get('sheetId', '')


    requests = []
    
    if sheet_order_id_상품목록:
        requests.append({
            "deleteSheet": {
                "sheetId": sheet_order_id_상품목록
            }
        })

    # if sheet_order_id_입금내역:
    #     requests.append({
    #         "deleteSheet": {
    #             "sheetId": sheet_order_id_입금내역
    #         }
    #     })
    
    if sheet_order_id_발주서:
        requests.append({
            "deleteSheet": {
                "sheetId": sheet_order_id_발주서
            }
        })
    # if sheet_order_id_누적발주:
    #     requests.append({
    #         "deleteSheet": {
    #             "sheetId": sheet_order_id_누적발주
    #         }
    #     })

    delete_request_body = {
        "requests": requests
    }

    if len(requests)>0:
        service_sheets.spreadsheets().batchUpdate(
            spreadsheetId=destination_spreadsheet_id,
            body=delete_request_body
        ).execute()


async def edit_sheet_title_by_title(service_sheets, destination_spreadsheet_id):
    spreadsheet = service_sheets.spreadsheets().get(spreadsheetId=destination_spreadsheet_id).execute()
    sheets = spreadsheet.get('sheets', '')
    sheet_order_id_발주서 = None
    sheet_order_id_누적발주 = None
    sheet_order_id_입금내역 = None
    sheet_order_id_상품목록 = None
    
    for sheet in sheets:
        if sheet.get('properties', {}).get('title', '') == "발주서":
            sheet_order_id_발주서 = sheet.get('properties', {}).get('sheetId', '')
        if sheet.get('properties', {}).get('title', '') == "누적발주":
            sheet_order_id_누적발주 = sheet.get('properties', {}).get('sheetId', '')
        if sheet.get('properties', {}).get('title', '') == "입금내역":
            sheet_order_id_입금내역 = sheet.get('properties', {}).get('sheetId', '')
        if sheet.get('properties', {}).get('title', '') == "상품목록":
            sheet_order_id_상품목록 = sheet.get('properties', {}).get('sheetId', '')

    requests = []
    # apply_rename_request(sheet_order_id_발주서, "발주서"),
    # apply_rename_request(sheet_order_id_누적발주, "누적발주"),
    # apply_rename_request(sheet_order_id_입금내역, "입금내역"),
    # apply_rename_request(sheet_order_id_상품목록, "상품목록"),
    
    
    # if sheet_order_id_상품목록:
    #     requests.append(apply_rename_request(sheet_order_id_상품목록, "상품목록이전버전"))
    
    # if sheet_order_id_발주서:
    #     requests.append(apply_rename_request(sheet_order_id_발주서, "발주서이전버전"))
        # requests.append({
        #     "deleteSheet": {
        #         "sheetId": sheet_order_id_발주서
        #     }
        # })
    if sheet_order_id_누적발주:
        requests.append(apply_rename_request(sheet_order_id_누적발주, "누적발주이전버전"))

    delete_request_body = {
        "requests": requests
    }

    if len(requests)>0:
        service_sheets.spreadsheets().batchUpdate(
            spreadsheetId=destination_spreadsheet_id,
            body=delete_request_body
        ).execute()


async def apply_sheet_title(sheet_id_list, ordersheet_number_title, depositer_name):
    sheet_order_id_발주서 = sheet_id_list[0]
    sheet_order_id_누적발주 = sheet_id_list[1]
    sheet_order_id_입금내역 = sheet_id_list[2]
    sheet_order_id_상품목록 = sheet_id_list[3]
    sheet_order_id_매핑모드 = sheet_id_list[4]
    sheet_order_id_예판상품 = sheet_id_list[5]

    request_name_index = [
        apply_rename_request(sheet_order_id_발주서, "발주서"),
        apply_rename_request(sheet_order_id_누적발주, "누적발주"),
        apply_rename_request(sheet_order_id_입금내역, "입금내역"),
        apply_rename_request(sheet_order_id_상품목록, "상품목록"),
        apply_rename_request(sheet_order_id_매핑모드, "발주서매핑모드"),
        apply_rename_request(sheet_order_id_예판상품, "예판상품"),
        apply_index_request(sheet_order_id_발주서, 0),
        apply_index_request(sheet_order_id_누적발주, 1),
        apply_index_request(sheet_order_id_상품목록, 2),
        apply_index_request(sheet_order_id_입금내역, 3),
        apply_index_request(sheet_order_id_매핑모드, 4)
    ]
    requests = [request_name_index]
    return requests


async def apply_formula_and_text(sheet_id_list, ordersheet_number_title, depositer_name, deposit_sheet_id, product_list_sheet_id):
    sheet_order_id_발주서 = sheet_id_list[0]
    sheet_order_id_누적발주 = sheet_id_list[1]
    sheet_order_id_입금내역 = sheet_id_list[2]
    sheet_order_id_상품목록 = sheet_id_list[3]
    sheet_order_id_매핑모드 = sheet_id_list[4]
    sheet_order_id_예판상품 = sheet_id_list[5]

    # request_name_index = [
    #     apply_rename_request(sheet_order_id_발주서, "발주서"),
    #     apply_rename_request(sheet_order_id_누적발주, "누적발주"),
    #     apply_rename_request(sheet_order_id_입금내역, "입금내역"),
    #     apply_rename_request(sheet_order_id_상품목록, "상품목록"),
    #     apply_rename_request(sheet_order_id_매핑모드, "발주서매핑모드"),
    #     apply_index_request(sheet_order_id_발주서, 0),
    #     apply_index_request(sheet_order_id_누적발주, 1),
    #     apply_index_request(sheet_order_id_상품목록, 2),
    #     apply_index_request(sheet_order_id_입금내역, 3),
    #     apply_index_request(sheet_order_id_매핑모드, 4)
    # ]
    requests = []

    ordersheet_number = int(ordersheet_number_title)

    result_입금자명_발주서 = apply_formula_to_text(str(depositer_name), "J4:J4", sheet_order_id_발주서)

    result_발주서번호_발주서 = apply_formula_to_text(str(ordersheet_number_title), "T4:T4", sheet_order_id_발주서)

    formula_예치금잔액_발주서 = f'=IMPORTRANGE("{deposit_sheet_id}", "\'셀러발주서정보\'!D{ordersheet_number + 1}:D{ordersheet_number + 1}")'
    result_예치금잔액_발주서 = apply_formula_to_function(formula_예치금잔액_발주서, "K4:K4", sheet_order_id_발주서)

    formula_입금확인중_발주서 = f"=IFERROR(SUMIF(\'누적발주\'!H4:H, \"입금확인중\", \'누적발주\'!R4:R), 0)"
    result_입금확인중_발주서 = apply_formula_to_function(formula_입금확인중_발주서, "O4:O4", sheet_order_id_발주서)
    formula_추가입금필요_발주서 = f"=IFERROR(SUMIF(\'누적발주\'!H4:H, \"입금확인필요\", \'누적발주\'!R4:R), 0)"
    result_추가입금필요_발주서 = apply_formula_to_function(formula_추가입금필요_발주서, "P4:P4", sheet_order_id_발주서)


    formula_예치금잔액_발주서매핑모드 = f'=IMPORTRANGE("{deposit_sheet_id}", "\'셀러발주서정보\'!D{ordersheet_number + 1}:D{ordersheet_number + 1}")'
    result_예치금잔액_발주서매핑모드 = apply_formula_to_function(formula_예치금잔액_발주서매핑모드, "K4:K4", sheet_order_id_매핑모드)

    formula_입금확인중_발주서매핑모드 = f"=IFERROR(SUMIF(\'누적발주\'!H4:H, \"입금확인중\", \'누적발주\'!R4:R), 0)"
    result_입금확인중_발주서매핑모드 = apply_formula_to_function(formula_입금확인중_발주서매핑모드, "O4:O4", sheet_order_id_매핑모드)
    formula_추가입금필요_발주서매핑모드 = f"=IFERROR(SUMIF(\'누적발주\'!H4:H, \"입금확인필요\", \'누적발주\'!R4:R), 0)"
    result_추가입금필요_발주서매핑모드 = apply_formula_to_function(formula_추가입금필요_발주서매핑모드, "P4:P4", sheet_order_id_매핑모드)


    formula_입금내역 = f'=IFERROR(QUERY(IMPORTRANGE("{deposit_sheet_id}", "전체입금!A2:E"), "SELECT Col1, Col2, Col3, Col5 WHERE Col5 = {ordersheet_number_title} ORDER BY Col3 DESC"), "")'
    result_입금내역_입금내역 = apply_formula_to_function(formula_입금내역, "A2:A2", sheet_order_id_입금내역)

    formula_상품목록 = f'=IMPORTRANGE("{product_list_sheet_id}","\'시트1\'!A2:M930")'
    formula_예판상품 = f'=IMPORTRANGE("{product_list_sheet_id}","\'예판상품\'!A2:M930")'
    result_상품목록_상품목록= apply_formula_to_function(formula_상품목록, "AA1:AA1", sheet_order_id_상품목록)
    result_상품목록_예판상품= apply_formula_to_function(formula_예판상품, "AA1:AA1", sheet_order_id_예판상품)
    result_코드번호_발주서 = apply_formula_to_function_in_everyrow(sheet_order_id_발주서, ordersheet_number_title)
    result_코드번호_매핑모드 = apply_formula_to_function_in_everyrow(sheet_order_id_매핑모드, ordersheet_number_title)



    result_입금자명_발주서매핑모드 = apply_formula_to_text(str(depositer_name), "J4:J4", sheet_order_id_매핑모드)
    result_발주서번호_발주서매핑모드 = apply_formula_to_text(str(ordersheet_number_title), "T4:T4", sheet_order_id_매핑모드)


    result_dropdown_발주서 = apply_dropdown_validation("\'상품목록\'!$K$6:$K$800", "D12:D941", sheet_order_id_발주서)
    result_dropdown_발주서매핑모드 = apply_dropdown_validation("\'상품목록\'!$K$6:$K$800", "D12:D941", sheet_order_id_매핑모드)
    # print(result_dropdown_발주서)

    result_공급금액_발주서 = apply_formula_product_price_to_function_in_everyrow(sheet_order_id_발주서)
    result_공급금액_발주서매핑모드 = apply_formula_product_price_to_function_in_everyrow(sheet_order_id_매핑모드)


    result_주문상품_발주서매핑모드 = apply_formula_product_name_to_function_in_everyrow_mapping_mode(sheet_order_id_매핑모드)
    # =IFERROR(INDEX('상품목록'!C:C, MATCH(1, (E12='상품목록'!A:A) * (F12='상품목록'!B:B), 0)), "")


    # requests.extend(result_주문상품_발주서)
    # requests.extend(result_상품매핑확인_발주서)
    # requests.extend(result_공급금액_발주서)
    # requests.extend(result_발주상태_발주서)
    # requests.extend(result_수량_발주서)
    # requests.extend(result_코드번호_발주서)
 
    

    # result_code_number_코드번호_발주서 = apply_formula_code_number_to_function_in_everyrow(sheet_order_id_발주서, ordersheet_number_title)

    # result_dropdown_발주서 = apply_dropdown_validation("\'상품목록\'!$K$6:$K$800", "D12:D940", sheet_order_id_발주서)

    # requests.extend(result_dropdown_발주서)

    request_protect = [
        apply_protect_data(sheet_order_id_발주서, "J4:O4"),
        # apply_protect_data(sheet_order_id_발주서, "T1:Z940"),
        apply_protect_data(sheet_order_id_상품목록, "I1:BA1000"),
        # apply_protect_data(sheet_order_id_입금내역, "A1:Z940")
        ]

    request_function_index = [
        result_입금자명_발주서, 
        result_발주서번호_발주서,
        result_예치금잔액_발주서, 
        result_입금확인중_발주서,
        result_추가입금필요_발주서,
        
        result_입금자명_발주서매핑모드, 
        result_발주서번호_발주서매핑모드,

        result_예치금잔액_발주서매핑모드,
        result_입금확인중_발주서매핑모드,
        result_추가입금필요_발주서매핑모드,


        result_입금내역_입금내역, result_상품목록_상품목록, result_상품목록_예판상품,

        result_dropdown_발주서, result_dropdown_발주서매핑모드
    ]   

    requests.extend(result_코드번호_발주서)
    requests.extend(result_코드번호_매핑모드)
    requests.extend(request_function_index)
    # requests.extend(request_name_index)
    requests.extend(request_protect)
    requests.extend(result_공급금액_발주서)
    requests.extend(result_공급금액_발주서매핑모드)
    requests.extend(result_주문상품_발주서매핑모드)
    # requests.extend([result_dropdown_발주서])
    # requests.extend([result_dropdown_발주서매핑모드])
    
    # return requests
    return requests



async def apply_formula_and_dropdown(sheet_id_list, ordersheet_number_title, depositer_name):
    sheet_order_id_발주서 = sheet_id_list[0]
    sheet_order_id_누적발주 = sheet_id_list[1]
    sheet_order_id_입금내역 = sheet_id_list[2]
    sheet_order_id_상품목록 = sheet_id_list[3]
    sheet_order_id_매핑모드 = sheet_id_list[4]


    requests = []

    ordersheet_number = int(ordersheet_number_title)




    result_dropdown_발주서 = apply_dropdown_validation("\'상품목록\'!$K$6:$K$800", "D12:D941", sheet_order_id_발주서)
    result_dropdown_발주서매핑모드 = apply_dropdown_validation("\'상품목록\'!$K$6:$K$800", "D12:D941", sheet_order_id_매핑모드)
    # print(result_dropdown_발주서)

    result_공급금액_발주서 = apply_formula_product_price_to_function_in_everyrow(sheet_order_id_발주서)
    result_공급금액_발주서매핑모드 = apply_formula_product_price_to_function_in_everyrow(sheet_order_id_매핑모드)



    request_function_index = [
        result_dropdown_발주서, result_dropdown_발주서매핑모드
    ]   

    requests.extend(request_function_index)
    # requests.extend(result_공급금액_발주서)
    # requests.extend(result_공급금액_발주서매핑모드)
    # requests.extend([result_dropdown_발주서])
    # requests.extend([result_dropdown_발주서매핑모드])
    
    # return requests
    return requests




async def apply_formula_and_text_onemore(sheet_id_list, ordersheet_number_title):
    sheet_order_id_발주서 = sheet_id_list[0]
    sheet_order_id_누적발주 = sheet_id_list[1]
    sheet_order_id_입금내역 = sheet_id_list[2]
    sheet_order_id_상품목록 = sheet_id_list[3]

    requests = []

    ordersheet_number = int(ordersheet_number_title)

    # result_입금자명_발주서 = apply_formula_to_text(str(ordersheet_number_title), "J4:J4", sheet_order_id_발주서)

    # formula_예치금잔액_발주서 = f'=IMPORTRANGE("https://docs.google.com/spreadsheets/d/1tRaty5BdhhVWrBvsx-SvMBHnnJaZZRekh2-4SJnFuqc/edit#gid=516836762", "\'셀러발주서정보\'!D{ordersheet_number + 1}:D{ordersheet_number + 1}")'
    # result_예치금잔액_발주서 = apply_formula_to_function(formula_예치금잔액_발주서, "K4:K4", sheet_order_id_발주서)


    # formula_입금확인중_발주서 = f'=IFERROR(SUMIF(\'누적발주\'!H4:H, "입금확인중", \'누적발주\'!R4:R), 0)'
    # formula_추가입금필요_발주서 = f'=IFERROR(SUMIF(\'누적발주\'!H4:H, "입금확인필요", \'누적발주\'!R4:R), 0)'

    # result_입금확인중_발주서 = apply_formula_to_function(formula_입금확인중_발주서, "O4:O4", sheet_order_id_발주서)
    # result_추가입금필요_발주서 = apply_formula_to_function(formula_추가입금필요_발주서, "P4:P4", sheet_order_id_발주서)
    
    # requests.extend([result_입금확인중_발주서, result_추가입금필요_발주서])
    
    result_주문상품_발주서 = apply_formula_product_name_to_function_in_everyrow(sheet_order_id_발주서)
    result_상품매핑확인_발주서 = apply_formula_mapping_check_to_function_in_everyrow(sheet_order_id_발주서)
    result_공급금액_발주서 = apply_formula_product_price_to_function_in_everyrow(sheet_order_id_발주서)
    result_발주상태_발주서 = apply_formula_order_status_to_function_in_everyrow(sheet_order_id_발주서)
    result_수량_발주서 = apply_formula_product_number_to_function_in_everyrow(sheet_order_id_발주서)
    result_코드번호_발주서 = apply_formula_code_number_to_function_in_everyrow(sheet_order_id_발주서, ordersheet_number_title)
    result_공급금액_상품목록 = apply_formula_suppy_price_to_function_in_everyrow(sheet_order_id_상품목록)

    requests.extend(result_주문상품_발주서)
    requests.extend(result_상품매핑확인_발주서)
    requests.extend(result_공급금액_발주서)
    requests.extend(result_발주상태_발주서)
    requests.extend(result_수량_발주서)
    requests.extend(result_코드번호_발주서)
    requests.extend(result_공급금액_상품목록)

    result_dropdown_발주서 = apply_dropdown_validation("\'상품목록\'!$K$6:$K$800", "D12:D941", sheet_order_id_발주서)
    # print(result_dropdown_발주서)

    requests.extend([result_dropdown_발주서])

    # formula_입금내역 = f'=IFERROR(QUERY(IMPORTRANGE("https://docs.google.com/spreadsheets/d/1tRaty5BdhhVWrBvsx-SvMBHnnJaZZRekh2-4SJnFuqc/edit#gid=322712525", "전체입금!A2:E"), "SELECT Col1, Col2, Col3, Col5 WHERE Col5 = {ordersheet_number_title} ORDER BY Col3 DESC"), "")'
    # result_입금내역_입금내역 = apply_formula_to_function(formula_입금내역, "A2:A2", sheet_order_id_입금내역)

    # formula_상품목록 = f'=IMPORTRANGE("1Fg76vk3lucUwwvmcoQJVb4KiMfjJSNx2VEFSdhwYFy4","\'시트1\'!A2:M999")'
    # result_상품목록_상품목록= apply_formula_to_function(formula_상품목록, "AA1:AA1", sheet_order_id_상품목록)
    # request_protect = [
    #     apply_protect_data(sheet_order_id_발주서, "J4:O4"),
    #     # apply_protect_data(sheet_order_id_발주서, "T1:Z940"),
    #     apply_protect_data(sheet_order_id_상품목록, "I1:BA1000"),
    #     # apply_protect_data(sheet_order_id_입금내역, "A1:Z940")
    #     ]


    # requests.extend([result_입금자명_발주서, result_예치금잔액_발주서, result_입금내역_입금내역, result_상품목록_상품목록, result_입금확인중_발주서, result_추가입금필요_발주서
                    #  ,result_dropdown_발주서])
    # requests.extend(result_code_number_코드번호_발주서)
    # requests.extend(request_name_index)
    # requests.extend(request_protect)
    
    return requests

    # result_입금내역_입금내역 = apply_formula_to_function(service_sheets, spreadsheet_id, sheet_title, formula, cell_range, sheet_order_id_발주서)

async def main(service_sheets, destination_spreadsheet_id, source_spreadsheet_id, spreadsheet_title, depositer_name, deposit_sheet_id, product_list_sheet_id):
    await delete_sheet_by_title(service_sheets, destination_spreadsheet_id)
    await edit_sheet_title_by_title(service_sheets, destination_spreadsheet_id)
    sheet_id_list = await main_copy_sheet(service_sheets, source_spreadsheet_id, destination_spreadsheet_id)
    first_requests = await apply_sheet_title(sheet_id_list, spreadsheet_title, depositer_name)
    await batchWriteRequest(service_sheets, destination_spreadsheet_id, first_requests)
    requests = await apply_formula_and_text(sheet_id_list, spreadsheet_title, depositer_name, deposit_sheet_id, product_list_sheet_id)
    # print(requests)
    await batchWriteRequest(service_sheets, destination_spreadsheet_id, requests)
    # dropdown_requests = await apply_formula_and_dropdown(sheet_id_list, spreadsheet_title, depositer_name)
    # await batchWriteRequest(service_sheets, destination_spreadsheet_id, dropdown_requests)

    # onemore_requests = await apply_formula_and_text_onemore(sheet_id_list, spreadsheet_title)
    # print(onemore_requests)
    # await batchWriteRequest(service_sheets, destination_spreadsheet_id, onemore_requests)


@retry(wait=wait_exponential(multiplier=1, min=2, max=60), stop=stop_after_attempt(5), retry=retry_if_exception_type(HttpError))
def get_values_from_range(service, spreadsheet_id, range_name):
    # Google Sheets API를 사용하여 특정 범위의 값 가져오기
    result = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_name).execute()
    # 값이 있는지 확인 후 반환
    values = result.get('values', [])

    return values
    


def filter_and_map(input_list):
    result = []
    for sublist in input_list:
        if len(sublist) > 2 and sublist[2]:  # 두 번째 요소가 비어있지 않은지 확인
            result.append([sublist[0], sublist[2], sublist[1]])
    return result


def filter_and_map_version2(input_list):
    result = []
    
    for sublist in input_list:
        print("sublist")
        print(sublist)
        if len(sublist) <= 2:  # 두 번째 요소가 비어있지 않은지 확인
            raise Exception("길이 3보다 작음")
        elif sublist[2]:  # 두 번째 요소가 비어있지 않은지 확인
            result.append([sublist[0], sublist[1], sublist[2]])
        elif sublist[3]:  # 두 번째 요소가 비어있지 않은지 확인
            result.append([sublist[0], sublist[1], sublist[3]])
        elif sublist[4]:  # 두 번째 요소가 비어있지 않은지 확인
            result.append([sublist[0], sublist[1], sublist[4]])
        elif sublist[5]:  # 두 번째 요소가 비어있지 않은지 확인
            result.append([sublist[0], sublist[1], sublist[5]])
    return result

import time

if __name__ == "__main__":
    # destination_spreadsheet_id = "1G1qctmwr3pMKrhMHv1hMbvXrmmCI1TsJ5OLqKKg7U8c"
    source_spreadsheet_id = "1AdHR51GZnnyGuLs25c5Al3bgWX4TN9EXi2KgKBrOesY"
    # archive_spreadsheet_id = "1tRaty5BdhhVWrBvsx-SvMBHnnJaZZRekh2-4SJnFuqc"
    archive_spreadsheet_id = "1ai-XC7bTUlbKx1UVBEfgG40nM3GEBL_P3cjpVfKa1CM"
    # product_spreadsheet_id = "1vXx5vFkPWRsxE5OgfaB4t2beO6QFlBy3qUwFjRb6gtY" #000~100: 1vXx5vFkPWRsxE5OgfaB4t2beO6QFlBy3qUwFjRb6gtY
    # product_spreadsheet_id = "1vXx5vFkPWRsxE5OgfaB4t2beO6QFlBy3qUwFjRb6gtY" #000~100: 1vXx5vFkPWRsxE5OgfaB4t2beO6QFlBy3qUwFjRb6gtY
    # product_spreadsheet_id = "1vXx5vFkPWRsxE5OgfaB4t2beO6QFlBy3qUwFjRb6gtY" # 411~510:1kiYHm2CLEvKxiE9Fv2j1F7vA4DDem4DROuU26d2RXBo
    # product_spreadsheet_id = "1Fg76vk3lucUwwvmcoQJVb4KiMfjJSNx2VEFSdhwYFy4" #511 ~ 610
    
    # 211~310:1XDBIK9o2k4AVMcW2gfieDZ4qSBxcTW8npw596AY6KCY
    # 311~410:1kiYHm2CLEvKxiE9Fv2j1F7vA4DDem4DROuU26d2RXBo
    # 411~510:1kiYHm2CLEvKxiE9Fv2j1F7vA4DDem4DROuU26d2RXBo
    # 511~610:1Fg76vk3lucUwwvmcoQJVb4KiMfjJSNx2VEFSdhwYFy4
    # 611~710 "1Gk5qHILhSSKL7u8Z4P6Ur7_bNIqTrM-C3EVl6PPPNSI"

    deposit_spreadsheet_list = [
        #011-110
        "1P9dnwlNRJeDHBJw54aFkgzKAP136VF6aUC1w0aKJo7k",
        #111-210
        "1AIi6cL1Raco8NMFwUaPWyrBRJRtrbX-VgbiEE7viy8c",
        #211-310
        "1yYkdpHK7gjpPUR3b96WAS3RkJjNEr-v9BulCJAjzftg",
        #311-410
        "10qEZMpO8Lzog-d9-U6kIFNsqtElltt322NUpuj5eazo",
        #411-510
        "120qPRz3jDLREEMCYUZJNo_-jBtFUcu2fOe_8tYz-IVo",
        #511-610
        "1GIyWxNODXIMEImKASOKQBTU2kBOdgGrXQElI8A6HAJo",
    ]

    product_spreadsheet_id = "1Gk5qHILhSSKL7u8Z4P6Ur7_bNIqTrM-C3EVl6PPPNSI"


    product_spreadsheet_list = [
         #011-110
        "1Mqg_M3ybwoT87vQ8rXOPnjCegvbgBvUoYcjo188MQKY",
        #111-210
        "16t8okayXoD3jdyiBFJ4w43fNA3o5n7rONfH8aVF7iWg",
        #211-310
        "1XDBIK9o2k4AVMcW2gfieDZ4qSBxcTW8npw596AY6KCY",
        #311-410
        "1mogmU5-1YYFVlXLKdJwqYnaLVSLFJFIvxKgxQZAFxy4",
        #411-510
        "1kiYHm2CLEvKxiE9Fv2j1F7vA4DDem4DROuU26d2RXBo",
        #511-610
        "1Fg76vk3lucUwwvmcoQJVb4KiMfjJSNx2VEFSdhwYFy4",
    ]

    creds = oauthByWeb()
    service_sheets = build('sheets', 'v4', credentials=creds)
    # user_data_list = get_values_from_range(service_sheets, archive_spreadsheet_id, "셀러발주서정보!A581:D581") #회원가입 연동 테이블
    # user_data_list = get_values_from_range(service_sheets, archive_spreadsheet_id, "다른고유값시트!A18:F504") #회원가입 연동 테이블

    # print("user_data_list")
    # print(user_data_list)
    # first_names = list(map(lambda x: x[0], user_data_list))

    # print(first_names)

    # filtered_user_data = filter_and_map(user_data_list)

    # filtered_user_data = filter_and_map_version2(user_data_list)
    # print("filtered_user_data")
    # print(filtered_user_data)
    
    # filtered_user_data = [
    #     # ['48', '1JScmTyIC8YKuRE867hRjiP2_eh4DAVmlJMw-zdhKw2s', '48랑랑이네'],['506', '1HgYKDeGrRGxwg-j-yVqeZe6sJY1ip5mDJQ2lj4jLPt0', 'SH프레시푸드'], ['513', '1_eWdxS4Xdykn-GbZTTyOdH8f41GEvwURRcisG-PI9XU', '황소영'], ['514', '1Z0BJDjKsvPOrYXKD93NyRLUu--ecxpNue05n_UPLJjc', '주식회사호신호'], ['516', '1VhHatV-asswfTDa9M13B8Orf2aDSmbdYWBzwrI4dqjM', '김성헌'], ['517', '133IbYEp-aXQH-ECojwEEak6LKRgPKraS0yqTSvqNvWM', '김가람(과람아저'], ['523', '1y6sJA3EAAqaSLPgZMWYJjY5t2IZLrZGMBTXWI7sroIg', '김지영(동지코퍼'], ['525', '1zyM4XOmz1hRUwLM-2WXHh5BaJ-TMowTA0abn8cd7fe4', '525'], ['527', '1RZUWJpKa9b3CiBFNGkgUwHV2nJo1BE1gs9Uoe1XZmuM', '정민지'], ['537', '1kCgT5DWV1CcHTLJldd6BgiNU-Ara1zkX1TFKDMXiSms', '바이티존'], 
    #     # ['538', '1c3mQbrzmSfS19tAA7MWtVyP_DoT47KI3MPKhI24TUhE', '김규리,김경률'], ['541', '17sd8bF7sHRXpj8cEfTJ_44zhn8ALvSqJ26hkZlpU5ik', '그록정산금'], ['542', '1sClq5Zx710IIh5GhHzm6SdIgeZGmHVbmb8rNetSytJM', '조성현'], ['543', '1fmpcNUCK3wxx1M5TvCt1Cr00wtAaFwoUwPIFMnir5xw', '543_최혜정'], 
    #     # ['545', '1iC421AvucDQacamyihRfIP5WymejCEOC6Fd8t2vkre0', '최정훈(조니마켓)'], ['546', '1xUh--GSPTPLmMXXyHLiJ6DjyL3J9BG3py3SeSGJHkcw', '김혜연(어랑,과일'], ['547', '17A2-P3z_NA1BEGUXxRPEXHmpzRgKCmeA3hDuUWyQ3Xk', '이철호'], ['552', '1zt-VXJNilmSDYDzEhZqyV12Ern3WVu2-IivHWSJ4H1o', '가순성'], ['553', '1SCC7A90vXM5srUZpubCyc02Px3RsCpOeUjC8_Nl5g88', '김지인'], ['554', '1F1N6d2Co-E7LbNfnuj_xfTcJczBdUcVjQugp8sUi_3o', '싱그린청과'], ['555', '1Pu9P4zFoDdFXK9ER_rZaEj2fa0FX36YEyCZIAV-YkEw', '(주)마니쿡스'], ['559', '1PLYZbwJy_4EqiWoQw7lMAC6RGc9f6-BEN92yZmAWRZI', '박지환(와치맨컴'], ['562', '1jYeZFz3CNgXkBV3z1CsoD8k_e6xH-vZa3M0KboBD8rk', '우리DC마켓']
    # ]

    filtered_user_data = [
        # ['22', '1RODBFbJ0p49qvpEeMOLLBtllLLUKZkxKX0Dax4V_-ms', '오늘의과일'], ['23', '1xa1tixBAnCPSuV0YMyTJvBOSqzR9T1M6zeymXGXXbYY', '김주원(제이네이션)'], ['24', '1e4kInWbkfgWtuvw7rZ51Bu2ipl8uq5s8w9D9oKkrAYQ', '24 과일남자'], ['25', '161KtREzfekUj3CP5WrhACAeYjspFQu4pognbztoy9Ag', '25_과일브릿지 ((우리은행-오픈뱅킹))'], ['26', '1zUII2T2o5rOAgldXE8dVvUiQy2Aitu7XkFd72KISfsY', '원블리스토어'], ['27', '1j4xwgYcvXxdAWfD5zWbs7iR3ZTqbw0vEfU09aVkl6gs', '산속햇살농원'], ['28', '1ye5ajpR0Rw4xi52eT_zEzX_tqPOfEkFCEGOpDUxbM0g', '노형준'], ['29', '1Pi7HXmaIJhHG14hI5ZoD_DpYKN5TkDWix55nyRH8M10', '심혜림(혜성스퀘어)'], ['30', '1aTp4LneRZScbMfDgRxixmZkICX2GzdtdQvXZvgcQCZk', '박정아'], ['31', '1wJtbxwKUmU8WcLGJpDbwcBmvWgYZJENAAXmwoPOSfdM', '김혜정(탐나는만물상'], ['32', '1FIgsgrK2_yNHSmW1jGGDKv6FVDjbRBaKp-vypHUXndI', '최경민'], ['33', '1SJ6Pn5VRWWlz4IvdIEV1k7FtoL7HN3qQVoKmSpq6HOo', '33 청산도실'], ['34', '1Gy1rC7SDAzQKU6RheshVjsfo_xlJf0V5AbC6kLlpLuk', '34바르다정선생'], ['35', '1FqzLkjwGTQp7l9qEZ9Ui5LcZDc3kj8h6cY55li1XmYw', '35(주)파인커스'], ['36', '11J4iB0i5gNlIbZVAgkyqYgqIvHihQG6d9oKw2x4yOLQ', '윤길원'], ['37', '1ITVWxwwQgqj01i4SizXPKSGgYwb4MDOA21UDbrvyWV0', '37_허니비'], ['38', '1hqFUQalfqiQsA8snWTh5ApsFW7pJUxImQgWUPWJFSkI', '김주연(달당과일)'], ['39', '1j0_UGC_PsWQ6jppeWykBktMv5pmJGPjbBjJqLVAJlDE', '달빛농원'], ['40', '1Y7uNXFetUaO3m1pnQulVj1uBv6AztaFBt-XE36t63XE', '박대환'], ['41', '1PZ6v81NYrknIMKH8Em09Uzb1jm1GkwpV6oPy_jPrOK8', '시즌바스켓'], ['43', '1QxOvUzXyvQfGPYFYL9dgBbfO1dXb4MlphOR2ZrVft5g', '프룻영'], ['44', '1BX2wMVUfRoMYvRzC0Dma6vsU84CR5RaRr8cLwObERj0', '44_과수'], ['45', '1E2XBdZiioiU5MV7-AKRuk0CYog21Vtva_reHnjJcY_Q', '머드러기'], ['46', '17K5GZXKTie63NE6mcDNpNg7vncT2llt72rVr6Od4yr4', '46'], ['49', '1ZPMS89eDfgRjFO-_EjMnthB0cOf80qbzUxCYo7kO1CU', '라이프by힐링스타일'], ['51', '1j4RGY92GMacZ7w8KMt_V-Ao2d0VLHFCdwHJx3sPewDE', '한상훈(플루토)'], ['52', '1TlFiE0JrnoXOROTMix0nWRIoupGWrYur5mMfEVQ2Hps', '매일식탁'], ['53', '18L2VzDHxsQ_gR7xCNO6G1Gt__OGaI7npt6UexrsOTDg', '53'], ['54', '1ZcOqToRl3h3mgeyDku-1u2CS99Qv1wBSIqksLzsfguI', '김민지(싱싱프루타)'], ['55', '1JbW1xWiNnC9pwlq-kOkwRcTMP6tV2sAW0PTOYWJnVxM', '빠라삐리뽀임서은'], ['58', '1Mgkh86B4FFQgRw5W_MwVcSeQA3w-EfG-BW9V5OYYCMo', '58_해성'], ['59', '1sQXSzi3QqRw4Mkw_eKjdDdhBnXVEeS9hUfYMqGE5Ibw', '59_원디트마켓'], ['61', '1ayjlGjPhH6G65ScAYgXVmhLd5OG6Z9fAKP3Iredisa0', '61_원가네농수산'], ['62', '1JFcsCTSRZnh8JJDbYH2trLtKX_wWp6bBvnrkMa7Zv4k', '이유영(뷰파인더)'], ['63', '1leWnT9R4-AMusEEKZSD7ItvAQubHUtaIem4bhox2Pp8', '63_샤니머스켓'], ['64', '1_laJw2PiYgr1nVMN0GHIbcANXm9WCaogqq7h_c_VDtE', '달콤상회'], ['65', '12YIF6GnENnX_oYWj8Nou9vifCeKhUdwrN3lS_Vwng3w', '맛나락'], ['66', '1pe6cqbiUYmOd12v4RHTMCudajPoGxi4dMaLE8Dui4eg', '이연승(맛보장)'], ['67', '1oLcBeKO3q7Kmf4o565r7iCQ-j6PAlOCRYbJx9HCxVa4', '파머스 프룻'], ['68', '1176kX8rJqadpzUDZcRDOanklZaBW3dcwHAZ4T2-yN0k', '과일농장 막내아들'], ['69', '1XWHkrb95YgyYF1zFQJ-vGqEggx-kL9veIyKAdIhxN5Q', '도대웅'], ['70', '1eD_exyqQA1oWbJLLu6Xt1qahDBdzst5wFHNWbYjrKxI', '과일은 프룻'], ['71', '1nTs7-18Sb2T1w-xbDtLYQ1UhjBHTb2Dpw3ROJFLBeh4', '김정욱(왕달다과일)'], ['72', '1uSr9p2tU_OxQv6fSBIzAhRZtb8vDIWvNqVBWU6ovlDI', '72_피어나팜'], ['73', '1l25uhivkbC3p-Js4QL5a-hvnTswVncve3DjxB4mn1lM', '농부의상자'], 
        # ['74', '1jRSQ71Hi686E6--8MVix3Lml9EWMRklH5I4ibnZPU0E', '푸드여왕'], ['75', '17vnpqSRm0hZ0c7M9Xow7DdiJ-aYIPyU8MMEb_a3pYPY', '식탐마켓'], ['76', '1RTS4gFbGpELMAoXn6gEq30U8Gxsw9sc759XAeLIalIo', '안서현'], ['77', '1UfwIKFVNNRT8VjN-hP_YWirS04TguC6HQqHTAYx770k', '박주현계절의과일'], ['78', '1gOsaEoEtS8z7HTiyi4GHWiPyt1SwxNrd55AzzcD02x4', '살가운'], ['79', '1-BzndUe5rc7DIU74Q6IsVqQ4yGMnxnF5ge0Pz1JsjKI', '달곰한'], ['80', '17aRoa0ZjpEMeJ9msNvlutnonCQlOAZgWiul1K24JAk4', '선희네농수산'], ['81', '1wYgtTYSHcq7YAm8XMgR-RmYDFtgAaKp684lK1eKfKB8', '81 꼬기스토어'], ['82', '1S3HcXMgbfe3vADUn7Tay55RCocrSfAwkMQKHhroei90', '또또네'], ['83', '1G5otw_1_zBtP5EFuIkatsYH-Hbchz9wHJ2Nd2RbnDx8', '83'], ['84', '1h3KvSL3h9OXTrm1I-9NyDXidAouldwS3DHPvE8DDdfE', '84월하소반'], ['85', '1T1TAX6v2oHlGA5HZ8cH-7lPhOGKmV11L9q5kPQgUM8s', '스마일파머스마켓'], ['86', '1oc0w-qeb5O0OI7ZP8Zn8ZQ7czwt9mbrj_dq7sAl9hXs', '바비라인'], ['87', '1R0kiSz-GIgoE2YlSZoLFaXHWboOwfQRkcK2rgORpepg', '87 과천꿀밭'], ['88', '1R4VBiPQDKtybRmZtLzy5THUaFErzmC4irw3SCOdwynk', '테이블메이트'], ['89', '1XEsW7sHt0ZJOGBGZKGdTYlc4bo80W2sZQi087Mnwu6k', '89_여우농장'], ['91', '13LwXgiUtZizbhxjD_Hnxyx7bF4MhNjidp2jD369dEe4', '디인비'], ['93', '1_33jO5Iuc7nVxFPifejAtBK5V3C4Px56T-qG6LoVkGc', '93_밍스상회'], ['94', '1Si6c_M-ZYct2LmUUsr8l36AQkhoylkJgn_NHcKx3Es8', '94_위베이스'], ['95', '1ZbAOBvR1psYZ6LqQY61rJehkixtPXtFDb4eKTUU0brQ', '에이투지플러스'], ['96', '1MTaWFHrQJprsmz5olKFm0iN7uLfbHNxhEYwOGoHfwxg', '아이스베어마켓'], ['99', '1nc6sZ_FU5yUETPVzIjBYQfBCbujbQthbaK7I_pAF0gY', '99_준형'], ['100', '1l6fT8C0CMLNi_iAkrK5YmW6jqpfihZtV2J34R8tB6Kk', '달이네과일'], ['101', '1w1UNz5nbxR2O_2-GLEhOqDllBgTaX1czMROkAV93iiU', '101_상큼달달'], ['102', '1t2qpE4H4D2oHqLDFYJnBOBS_dyZ4CZmGKWLlVHsb8Hs', '민정푸드'], ['103', '1LSXmHjgYcqSTcL7sdolwvMihHScZXAjy9Rh90OJavn0', '103구쁜'], ['104', '', '주인없음'], 
        # ['105', '1FPRUzUjvrbZ7wscG9btA97Tk4f_1_1nk3G-RLUPJ--k', '105_달콤이창고'], ['106', '1GQErkm1PDDtGaxhObrFEWWfN67kA-Kgb2h3KUbxyvmc', '과일맛집'], ['107', '1So-lQEax0X6JUM1U92pc9HGE82qzvcGzPnv42BYc8DQ', '오진담'], ['108', '1hqmjxkPlodSQpCxnksMxppp9BItuRvKWlaPvke9Ulwo', '마인드밀'], ['109', '1IefzlRRh3NHlQcJWIK0ZTJyYK3PF-cat3xBjqDvGboQ', '함지헌(케익엔아아)'], ['110', '1-J8oUE4IUwOvFkxN_2ZTa6AYxFB8Eb3Qw00flde-UMo', '110_미니미홈'], ['111', '1D1kwyNNtATDmhAq6tT2jHUqhaOrhC1lf4FE2XAGkiPQ', '포항제철과일'], ['112', '15aiQxfUhDdLmV4h-GMz8IWu0EMoCsv3FhxKDgNP3Fs0', '112_소미네팜'], ['113', '1txBiktgHE3QFIMS7oavpJU_wql8797NrIUmtRLeCG9I', '112_더프루츠'], ['115', '1SKJSun332I4X53RfTYmtpQMnP4jF6lEWgm5kixX3Yi0', '115_송지선(원씽'], ['116', '1-iVi3ml3qxKxiShcEaGOK2oEzT_Ea1o3n3EPc_poSvs', '김웅래'], ['117', '16utA5lzrkiGlfrOrNOWVMYAZ2ZYzcwXdg6aSpGHQ_X4', '김서영'], ['118', '19Ap6hGSR0dsluAVtA76NEzz7fX8ZpF58o0wJMP7Adzc', '118논논'], ['119', '1dqk-ef0O3OaCHqmQvzZnIgxZX9hwcHpaAhvseCmIHTY', '119'], ['120', '1EnauQRZmXQm1AMpkl120fML1h5zvLCNXcMWFjy-0bvg', '3ishop'], ['122', '1DR_f5Cr0YMfdovRXUTXv7LsVcyxDaDwklQrhQ39qDA0', '달보드레푸드'], ['123', '1GNMHvzAaubPqWwpR51LfxzdzSySDXa5bpytbzlU48a8', '범돌농장'], ['124', '14cBUnPxJodEATmBOpzjlVqLC7Jjy-XWeVNZWnSBPhzU', 'buy피플'], ['125', '1k859lG52TmmMlPtJb5FWXKnrFsc12TyUhKPLQkS8jEE', '더반프라이스'], ['126', '1TyBxoqjstXCHztMrBq8rjjEsdgcRIlDtV_cRezGfNwc', '샵엘제이홍'], ['127', '1G4k91F4Z0jphI8pzayNSof8Hlzk5h5YIq5sZ3d-SDog', '남승원(나라나라)'], ['128', '1OSyTz2DqlVyTz7EVH1AHpMjiblfOqN_QpGwjmIMAKqs', '128_1도씨'], ['129', '1mYLKvqj1_zoIfTUjE07Fiqdh7uZjxgyjO8AivB4mofA', '도담토담'], ['131', '1o44LzTuaJKnBRrEZ_E80V7TzptWRU_EyOjCwloN-mzU', '131_남이네과일'], ['132', '1aJC7kiY-5toY8E3lDYIIpe6p5p45SuQdBWDAV0g9ViU', '미들상점'], ['133', '1epMqOVp3YenLJyPWeBMJ8TrFk38h6CRI032hKJNjT50', 'abba'], ['134', '1Uw-BGUDRxdBtiNDywYPaZMBPDgMWEodQnK7LdBaAMNY', '134_박상완(피엘'], ['135', '1Cwq5yReEdYR3_ZTgWjWEeVKmWwgqHFKAK2Fl3wN-KRI', '설월농원'], ['136', '1xUVL58bmnMV154jVf00_m4w2zFiTwwPhkR7uStBBshM', '판매왕'], ['137', '1BXImg5y6PaXGHI69MMswmsPBFSmJ4U09dK77m4qzFEA', '과일은달다'], ['139', '1_eeLMdq76Q7iqoD2xYqG-KntOmDuPQPACwdU8pZJhSk', '139땡땡마켓'], ['140', '1IC5D7wjgVTEK2YnSSbFMZaKQFjlWlAbHlcZ0yOr5Crw', '최호성'], ['141', '1cxl30yl5eZ3qJE_P6vFx1GTCWylzqEwS-sF_BTdShRo', '촌플러스'], ['142', '17MUqjcZVtUujcYMTpRDryN6dpIXmWKIYkedrkyrrEf4', '자유상점'], ['144', '1BDggipZEOJs9ck6zbzV2T70zXbk-oiUb2nhoArwsdMk', '니즈올'], ['145', '1fAv0SXKt-lUTthNEiLK__3pM71IIxwUGSeT7srUfK7o', '팔뚝이네'], ['146', '1D-Oaf_EBYDCFmMTe5N3R5DLxwsrydovxpUTDowy3ozY', '146_주인없음'], 
        # ['147', '1-k-iiZHkm_NSbt3DTjj2EHYApUvpZwILzmaoJ75V9BA', '본스토어'], ['148', '13x_vWQqsFn_UNQKHEqM8qqXUPoOgF8zaKoObPU-24QM', '148 스마일프룻'], ['149', '1DWjrgohlyQQRALxKLGOSx6-FNl_bxkZMs6nV-TbK-dU', '149_플랫팜'], ['150', '1J0k67ytGgVRlT2gb9i3T7c-YCtHJdWDXmqF_rVGFv0o', '150_>비스'], ['151', '1kaGgrbi57_7wOVuea8n0udLCo6lRasprQ0TqcoF8myo', '마켓꿀비'], ['152', '1vlPRO7WKS4p1FX7Ka5qcGzWR7D5OME1wiDqi8sLKISo', '152_수족관'], ['153', '1CtGr7TpphzT0AVmmGgOlg0uYLHuRnLFtf2wDi5q3Kkg', '로열다이닝'], ['154', '1HtorhVqFaF6hQJs89mfTGYekrNhKQ3PPKcOzjgeVw0g', '154_프루츠윗'], ['155', '1Jc3c2GzMXhtuFHpjf0ntBGSFsRbgaPiv5gS5dPoNuHY', '부자농'], ['156', '1YvuRWpzt1oq4smwJ2ASn3gohdvmBsYSJx2lI9ttdze0', '다맛나'], ['157', '14iLoh68WxK6A8S5mPt7VzP2FyS_Xyzz1F1_oRB2vlw8', '이레네'], ['158', '1Epzga4qPXFgAPlRdmi3UKLLgyAz7Op6wYdi9fiOXcZE', '압구정과일언니'], ['159', '1OcwxhHsKsJ5cZvezUSlkBxcyoAl9hZ4mrUMq9wCTYHc', '159배혜영'], ['160', '12ZFQnUPRp9sfxyXxa_0jWvWdefZXHG8PMJS1cpgMqWQ', '뭉태'], ['161', '1xqm8ExUh9i9lLrKQDWYeEX11f8ao2ARFfXVf3tGHaH8', '베리그뤠잇'], ['162', '1V1BM8YKvKemU3sns069Bn1P44Cfk21d6yJefZKvytfc', '과일공주'], ['163', '19nDEB63qJ21ne4Rkq-PrBRgLua2Fi2FcrxJ-4qXeKOU', '163_팔도팜'], ['164', '1Abs5QSYaC2V8x_aBCiP21NQymm-2eucN38ipi1z8rvE', '164프룻해피'], ['165', '1hZ1JlREputO782Cl6a1f7uEvgPYm7vhP1z47tgrMfNY', '윤지현'], ['166', '1Knkg1uchCIiJPilvmJf2BrTUn1nH6cDjY1DmGgkNdWM', '166_모두밥상'], ['167', '1iQJJzyl639Rh_iK3Xqyc60_aflHJq1mpe8cgD5TTwd0', '167 과일청년'], ['168', '1xQR_uanq6166NBnAWdRQTOCQlrY9Ivs-OFoYjM9RPvQ', '철이네'], ['169', '1xgVM8SZ1Aac8ArwB6WREoATH-TnXy2fx2Vl6_8AESbk', '다맛나마켓'], ['170', '1NOn7-kP-srvXAxd1ePNRtBgZAJNKPjeFuBBTWN7mXGI', '보답'], ['172', '1gRC98XO8gfpvk8e9vQzIY8LMMQZbnhqcB-J9RmAQ-Ko', '식품의즐거움'], ['173', '1QE7YHQplxFyaqrfduOJR-HfbrDKns8hOFUfLBZznkyQ', '프레쉬영'], ['174', '1t5gXpXwLUP25vh2rPmyj7DXxCIJPBCbwjzApijIyQQE', '174 크림프룻'], ['175', '1r8h8t8PGUAcSy8OWXem2okcTdL1h8Xh24NhwD-CAbJ4', '하박과일'], ['176', '1HQPXWt7tfoeo-UNztnPtN9MpyqZPm4cUxqCoqaKcY4I', '킨더플러스'], ['177', '1X42aZTspAwZkUSPpkNcSsqi2wAdu4-sxmPdVx0BZ-S0', '신중히엄선하다'], ['179', '1pCZimr_EYGfi2ihHDi2LTXAyVVPoFQRP8moxEEP55Us', '179_주인없음'], ['181', '1LL1DZ_vNP1WGPQmDxvgFGC3GcFE72_1mC15qGXPQiMc', '181_주인없음'], ['182', '1XpoZdGWYZrAbkcsWsi4heIx9qMdQfmAS3Q0UCqR5ROg', '182김유림'], ['183', '1Ufws_ZDb3EWakQH3qWHpYIwdR8tFc3kctr7QggUIIng', '박지영'], ['184', '12gK5LTMeUwbONK-cLrbm5MP9RlWy6ziK2rsuI4HcVa4', '184 이예찬'], ['185', '1Kb7M5P4osZ6XcDV8MVmj1zN-jUTcn0Uvjs1ZCQX8cek', '행복바구니'], ['186', '1Q15pQr23lHYXgDVAYUkn4q1AKu_TJ07GN3RhxniNfXQ', '186_애머티스트'], ['187', '1Ps4zshHVX5GkCmpxJMOuUhQIcve3PvXU8GIuCMNHTpM', '나는된다'], ['188', '1i1ua4lyiN9khQE9XCTFoPsz89lSdkooHQdgfzdnvVpI', '팜앤푸릇'], ['190', '1Tm1G0REDUaJCtWbejXrNAEdRWv92m69mJjmpH9PgXCU', '190대봉'], ['191', '1qJt2lgj-ruVRq6Z5zqCCy7CX8B08OgfR6wxYP7Ar5Z0', '최경숙'], ['193', '1IkTQ-yvNnLR04-zIkbD8fDYXga_S1lz-_zIgTplNhMA', '빅범'], ['194', '1JcZ1hl4nBIO0vsAOAn2LEGV2shcjbBExMi3UUia5VA4', '194 푸드향연'], ['196', '13e5ms9krnkPH5p7J6YvPuu4baqZ8YqeajU5gft5PVQI', '196_주인없음'], ['197', '1a2iaq660Jh0mK0piVj3mAl8NUenPWslULMhDlkEjZ7M', '197'], ['198', '15sKc1_sjgo4WV8LIF_M68RqctBKJeUcO4FVh1cv-mCs', '198_토박이'], ['199', '11jQJqw1MBWhK7swkz0wnFO250OGvFNCfOtL-arAznhs', '수상한상회'], 
        # ['200', '1WPQ-Y1MGS-mtz2qTvq942ab0PjC3f6MSJL_9WchnqqA', '달콤한꼬맹이'], ['201', '1s0nr4vTmMgYw1tkBgNBX09rruS-Zq7Z9iMESyRZOzc8', '김은애'], ['202', '1NJgK1M3-J5GBpvIAcxxkvh-KCdPE7Q5i4XSpe1L8urQ', '202'], ['203', '1pWxAluQWZwUW9nAn5zrqbZRa2fWKjd5EKbcPiXnUuhs', '203_주인없음'], ['204', '1Fe02qdIyeWyx8u1h5nr-CLw7qQnOBGqPWXdqiKE3pyU', '서원주'], ['205', '1y7-rkU49LqdYJLNdOhG_iDsHUnYINRzIcSSYLR4Gjys', '205_프룻해피'], ['206', '1DIpZGrRPMGwTyhGC_UOYs8o5d8x01uomUCkEeDJAspc', '티앤씨'], ['207', '1Kg5ZrYSQrDM_7ArheCrlufT-vsIs3fA65xJw76mO8GE', '아빠네'], ['210', '10B4mJWd95u3k2aBFuJwZdYbxcCfyhGctcJm0vcDnGw4', '김주현'], ['211', '1ZfZAfDQHb6XKGKts3s7mfTLhuhPJh0e-wVhgsC2qkYQ', '예니바미'], ['212', '151_OwzLC0qOrleBI6XvcpNsEqGdZi5swdXrBaz10xmI', '거치테크'], ['213', '12Kj2obs2kYJfLv9xNiBUXwOYeL6hTSS-l4NfYIxfY-Y', '213_엘도라마켓'], ['214', '1mieD7Lj40tZ7cPYBR_AEvcVrdCdApIfBMl2rCN3ODi0', '214_김소영'], ['215', '1Gc8-mj24ICXWRiaMktkS48qT6PEUmQ8WqpD8Ui2zmEg', '남혜식(남브라더즈)'], ['216', '1__-SRb0giGMJ4yGb0e4bLAC_5zbDPVWb-lzm8bbWMhI', '과레카'], ['217', '1JRmUZR1w_1L9rrUaoauGxooPTgMofR1pqnHVQmwuwjU', '이기세무역'], ['218', '1-JsD0tXTJaHsyFxFWPcZbIGks8haPB8PWSRiYhGKLaw', '218_바라던'], ['219', '1NA_UC5Ibaf8Unoy7vHalfJEbnBWsfSojBNxOvbkbav8', '프랑팜'], ['220', '1tPowpSJ4Ea15Q50tJtOBI3ztl5RIzqMB2btLv7X8g6A', '220 굿무드프레시'], ['221', '1kq8EKhDyAaA7ng5kgprjXefRuQRAkoXbZrlwy4vTwpM', '과일구독'], ['222', '1BTqhx2OuzG6gnmVpKIoUFOy4DcwrFxbrYE2tku5LdxQ', '김연희'], ['223', '1mt_zKAqLhvjQYkv_zUPI9zqFuPpsj1sDp-6WbBsgPFY', '223'], ['224', '1ZT2idVc50QtFieNm_IvMoGdTRCCujkaTSKRrq_kvGmg', '김윤후'], ['225', '14ov5HTlm-1Y9XMNHSuz2cNbmoGTa4fnsUJOZS_sMQJs', '225_주인없음'], 
        # ['227', '14KS38kY_hZD2061EmIaAm3cA3Rw9R2mEDoIczxdk3sQ', '227새콤써니네'], ['228', '1hrmwW1aznOQJewXDlyCsfCqEvsTjh_BXXVSZAqwH2r0', '예스마켓'], ['229', '1aEZw4wsj1HXCuxvcc5mnvJ0UI8Qkuo05-zmYPzblSWc', '디제이 유통'], ['231', '1CC5HAoulfHpomxSJWU2Zr0doNLwZQYj9uMsnk9YlUIY', '다마라'], ['232', '1iKJnf9JCFOvBUqcufaX9xs2YVrqIpVGTxFruIP6wwhg', '이름 : 이미경 회사명 : 글리밍 발리'], ['233', '1911UP9nR65fxxJO_Na8886T-Q-FBPvMaRs-twm4KsoI', '정호성(메론메롱)'], ['234', '13gTVm1GQEnrQqgc_Q55M8cWyrWrUAmt2gt_v7Ltm-18', '박인애'], ['235', '1wKg9_xHUVFRujWUsX9unEN38pD8yg3B_c1f8z1C8FI4', '정이나'], ['236', '102YTIjanaJqeckd-zJ_s5d-EgUeZQUB1F7G9wWw_-6w', '해룡산업'], ['237', '1P3Z7nF8kRh0H84yk26Y3G4EzfY_AWRPdU4mNyppTNaM', '나는된다GO'], ['238', '1ywXZQV6GXPP-xDAfAJ1Q0AqBD4TTTV214qOvUF79_aI', '김하얀'], ['239', '1hAdI2qGPe1UIV-8qPLBGOk2XLvSwub0LSEQBJSL4XXE', '239박희수'], ['240', '1wvqrK1DUrey1EVr3vOY5KH3vngeZHyT_aZKf1a7RAig', '이현주'], ['241', '1gADqw_nKmZGBWvkRGB1YbfxMdiDpCGGFX-KMNyLVlO4', '김향유'], ['242', '1ghDdb9oxfzMtlFcf0UJ107tw53cS54xaYcyMKLBMAlY', '박주영'], ['243', '1Zt4SVQIelLhPu-x24A1S3-t5YxIHhp83_b3mAd0AfvA', '밸류에디션'], ['244', '1ZJsmIsjhpXm2XfpLhjZr7tiFu1zjtbXsCR6Qb5wqMWg', '함홍식'], ['245', '1Aq48MHnTacmf6IUIslVp__0Yy7rT6xknlHuJRS38ddw', '주승현'], ['246', '1gszFDCCxVhodjKOiwBlGMy923dRM291C4jwX6fmF5IE', '박선영'], ['247', '13kPhDw3BLLG7526upsUb2EmDHIKxltN71CsA6Y1leQ0', '하늬로운마켓'], ['248', '1vvfXF0PWR1s91PcsWfGlEBg3dL6fMm_t2fhz435Vwh4', '한수아'], ['249', '1o65cvzYHsYXewWZ_F8V32wIZ8eR7lcRED44z3QkolIg', '쿼카상점'], ['250', '1EXkviQkdwUcxXaJWIVDmNlMyGpWbSdU4ZEZWVcmBCE4', '교동'], ['251', '1qZ7QJwDCfDuYYkRTkJ7O2eACJJcIx451It8ABDuave4', '유보람'], ['252', '13mkxLmEHSExlGrGS8N9A7X4uK0I46GGgvf5346RG9mA', '용리연'], ['253', '1xb7HtpQPHOc3c3j8l4Q-Aq_NwpKFmv_sJHkE1hGbYh4', '반용균'], ['254', '1pwQqmPdZnILJUcXY5-2fY1yOimJRz0lWFK1togzStbM', '150market'], ['255', '1sHXYfzFIRueHfpCQLnOnn9WNOKLS_MNcPBZN92hHulQ', '에스그린쇼핑'], ['256', '1mLWaNAB1yrKVIY6PRtTv3hTCuQAQMlgOYrgTziOGfUA', '영천파머'], ['257', '1bhzRmCtmrcjraE3Tuw8fv_5jQWGj3NI8INjQgY-wH3U', '팜700'], ['258', '164jZ_ebAh_SeJ_MNnH4la8ZRSOUzIw8qlNnaX-eg9eI', '엘리샵 710'], ['259', '1SB_p-YSE_Dsh17jBY3aWVP14dUJt8vdsYV33niPe8zM', '리브드릴리'], ['261', '1-26PbFTHeDXxaawArrnjQ5-S3OGSQc6MlCy2MpeSiNE', '젤마마켓'], ['262', '1tpj-ih2nTQ2AMmhlm8yd6WkI69u6HTj6F-4y8Kx9RRI', '위즈마켓(디자인위즈인)'], ['263', '1Cf7g7wNO6VoHgOs5FJf7X9LhGJGsOTxjIXIdUDs0t6Y', '263_팜700'], ['264', '1LQYXWFVEimAK__ARegvNMTYWl1Wc9Al8Waehex5SEpc', '박민영'], ['265', '1KHFHghFeSeb7iitVLkJLSsVSVj6j_rLFx1_vga66Svs', '한혜빈'], ['266', '1GecQPz9t2wPmzbRI3qcGbyxOX-VyNXdhXXFbiqLfM_M', '감(GAM)'], ['267', '1uXsU76wrxiOCaatWbr0IkfVrptbOR4bhjqmBw-qMENE', '267_김도환(청년'], ['268', '1ev-Zme1lwmT_zs1ZH0ZHBgW_MFrAyB7tu6p2XgIVTJM', '마로진'], ['269', '1AzzWneonNdwTKCpLG5p6RAGpGIrC4e8fFek6AYOaDX8', '시크코코'], ['270', '1MsEFUeFvoqSdHnKOTpNhVTkwahUGky0Ce50xskaO9cE', '하이뉴드림'], ['271', '1bvFhoOsLkgrZ4EgI3MTT2rzNb175ZhH4i8woMheRYP0', '주주'], ['272', '1oyw6bzOY2Mm2NHdhA222-bJhPTa95MOT7gnClYZKIpw', '272 호박스토어'], ['273', '1RTKqIUg3RRBaVaG4dw9piZKn4mTtIOmu0vajWBXU-qo', '시골가게'], ['274', '18J-VWBnrYCEgtReZYdfRPRLL4xQh9ajlZlaw5U82Fcs', '김재훈'], ['275', '1yobMzglC1nw00SX89eRdQ9pRUOMZvDCwsrrC4TnPUuQ', '위즈마켓(디자인위주인)'], ['277', '1yX6qBTucOtr4li9PDgI-u_QDKRvv3-1RZkaqm_xDRz4', '고택형'], ['278', '1O8GCeVRD-Gl3D13An4KbQ-_W2njqo3KtrBH2EMUXKm4', '라넬라'], ['279', '1FJ-A4njR-QK77qvnmP7E6EGISloa3Op6_l1bj8K8NGE', '박윤선'], ['280', '1M1CT01ZIJ5q_BZjxTtOJQfDb4Xl4oi5dZUW78C39PIs', '포동포동 농수산물(260 중복)'], ['281', '1385yxVrxmqXJxXOgU0hTzWrmf11qVlZVtJmcwIVM1pk', '대봉'], ['282', '1GUcwpeM7HFPZ_GDbH_3rQxUc-sIZGqm213YCrIxJXJg', '수수온'], ['284', '1n3FKR-p-XTaTZiZoSNkpLOeVA2X-6b7JanriOQhqZVo', '권미라'], ['285', '1X4idsd1bYe8C_SFNK0ytnyGljmbZylsJNUvHyACYVaA', '월드글로벌테크'], ['286', '1Q25PZQ_HoscEO1VjzqUtMQtYqsv2SdDaX94dEsAEQ9Q', '이혜련'], ['287', '1B9MXjpGAIr2qQm0KtjCSx9FchIGHb-9qGApuFN3P7_w', '김슬기'], ['288', '1bnowlZeFhYpGuJLiGC8RbFxwN9aD3_WV5eGJyiuMx1Q', '코알라유통'], ['289', '1zc5hj9_HdxFp2OQ0GW691Etwy_w3zeZ8SGccEWz1Zmw', '영천파머(256 중복)'], ['290', '1HHN35MYYN5r_KJPJOZnsMdO0_ZpVgDASkUa1w0t85Cg', '애드냅'], ['291', '1gMJy_Tvvk2cdk-QP4AtfIbJRFKGZWmi520HC-kiRkTs', '오트컴퍼니'], ['292', '12pGx29h8MgbQd4uVBOWWUKUskjV53kOUifdH0ZM0sPI', '노영미'], ['293', '1nFzi_lvljA_c71iyf5R7xBJMi7LKSU5uJ6V6mg2sjzM', '천왕봉'], ['294', '1DKA6mUxe8xyFbuJKCtEMBXkLq7qsl_iRywXw-ewamBY', '(주)인포씨에스'], ['295', '1opsRF3QC_2qC-Qq__FVKjXONGMjvzeV2563PPHe6LyY', '후르츠봉봉'], ['296', '1YpJo_dcXNLypwKrffgitvjm8cKJa7bVCYTg9ftmjALk', '이주연'], ['297', '1zxbvjbD6odFTuMYah-lA1vowu83SuZDXoqcg2FGTLrs', '주식회사기적'], ['298', '1KHuo78QuvyXL5_QtDl9YAEz11eWx0QBFgwmHsDtqSAw', '김서범'], ['299', '1LAfSh7CeOt3UdnGZCGN7TezOLoIFV1fV-LkhxQdhews', '최수미'], 
        # ['300', '1GezlWob8QumkewXl2GoTju1q5drvU9VgzkoroyV39PA', '디네로'], ['301', '1FyitvSjgSb4TyO5ZQPxE2x5GFwTZNewM4nn3A4Fg4wo', '주비오'], ['302', '1pTLDIMNJ56wMfA0iaDoUAxbUa22gFzOJ4xMku04-5Wk', '사계'], ['303', '1ZwjMv8qE5LX4Y1llHLFRlnKh8wY80w4u3mTLj3ZgnUc', '이호영'], ['304', '1DDQofm8jZHbpYY3axka5Ll12GwFxI90F4b-LT2vt9G8', '무릉상회'], ['305', '1hRtdyG426l83CCnczLWDuvNQnf1fzN0W0UdpRqtWiPw', '세컨드림'], ['306', '1eZVLizpOj0tQScR2b75h5eEUrvx-mn5FH7R9Lj38k40', '최희원(핑크펭귄)'], ['307', '1vW0JmzuqZj-h3JmrzWUEu8iAnI2ItNA6SXwsJckhgmg', '김호진'], ['308', '1FnOByXboCct8Xa_Z7LKSgFLHA8f_JeuqYU8c9JZIs9Y', '유재헌'], ['309', '1XzP1h2TrHmShxvO3zCI2977enlncxglEoSrgGBZVheg', '이지은'], ['310', '13x949_4YhH0DJpWA_bP4WasIGSsYqP_bJcWmRjUTFqU', '310고센포유'], ['311', '1rtoLCB0aMgOai5h9Tq7MFNZ0HSJYtibWIkXuhS2Pr-g', '핑크펭귄'], ['312', '1id9Egfo1IghdDF9veQtOt7PxaIfoNtKympXPZ-1xiWk', '온별천지'], ['313', '16o_86YsaxuIfeduiz9x5_fhW820GJfwzDJ085DWHYc0', '영준'], ['314', '1HHmowyxSOMzACV23tMV13l7Yn2YsuENbmQ05xP9DmcM', '엘케이컴퍼니'], ['315', '1z5OQ688lFn2nS3LIRM73ZkRBxnPirLoDfueM8xna3mg', '모든컴퍼니'], ['316', '1Kzfkcfh1Fgn8TGN6UhlTT9m3IiIZ6Bo3lr_5O2jrzXo', '316_이혜련'], ['317', '1haCXQFkgHcNmqm0z6Up9vA6H4-oYCsSZnoj42M8spUo', '여수39수산'], ['318', '1_4NA4bzM3zQ3JBVWG8QY4Je0BN_dWMxS-JTqCKMIb84', '고대현'], ['319', '1Jo1AeLeobg5DYMMU754M4d07_e_LT4lP1qUmd7ic7RY', '카페인컬러'], ['320', '1h6C0M3x2J0HubAf4Vezbs56rIHekcxQ1OohTmQ7F2ko', '프룻매니아'], ['321', '1vVrVXYE4bNOSGeOXfSRcXTjnlqFUzaarJ1t91GNhk5k', '미니샵포유'], ['322', '1Q0JXOZ2jcK-rHNt_Vpr3IZkvtxH0QzQybdZHInQ6l9o', '322대봉'], ['323', '1-eQ1WqpL7XQYRsiwt84zEtMIJqcn3heAgOYPM0xox_c', '이화영'], ['324', '1dhvxWSeL5GQMtwnFPDVbw3_mRvejsHT4O-1Hstz5lnM', '김지현'], ['326', '1cWn56c896DKoE3nz1mHd8dnUrUkg9CIXpZAWSKxcs_c', '326대봉'], ['327', '1X3LThRM9x1oKRBm4qTaZTUMxNWo85O_0NX3COZ0HOc0', '327 김주연(모두'], 
        # ['74', '1jRSQ71Hi686E6--8MVix3Lml9EWMRklH5I4ibnZPU0E', '푸드여왕'],
        # ['105', '1FPRUzUjvrbZ7wscG9btA97Tk4f_1_1nk3G-RLUPJ--k', '105_달콤이창고'],
        # ['147', '1-k-iiZHkm_NSbt3DTjj2EHYApUvpZwILzmaoJ75V9BA', '본스토어'],
        # ['227', '14KS38kY_hZD2061EmIaAm3cA3Rw9R2mEDoIczxdk3sQ', '227새콤써니네'], 
        # ['329', '1q2wxUJ-PXP4NOTNUYBPogqozFwPFGQ3OHOXB05BNBnY', '샤론'], 
        # ['331', '1vo5oaazGHcJKy9GARaOxDlAvzOK5DUz0Bed6fJ_A_0k', '송진주'], ['332', '1K7kAgxF8jY8F3UTe_BgGYrql2ZaXHKLg08P273ThmkY', '현이네만물상'], ['333', '1jlOfxuccVaSvlPAGAiYMaBqWj4tNdfags9ft8EIbVR8', 'KANARAT'], ['334', '13Lr4n9Iz0hUUpP23N-ND7CqJ1v6S8Hfz0ynV9NzL9tg', '이진영'], ['336', '1GS6iQRFxOJ49_rynFQoriftRwkSdJmOPHn30huYUSZw', '신월드마켓'], ['338', '1RAnWFmNvlv2VUk_biNbpUyhasr6o2Gt5HXZEk216sDs', '338'], ['340', '1oUxwsthVA5LKJvxvjSg5uRytprGanLiuUHsLqVzJ7TM', '모두얼마켓'], ['341', '1KlkCuCHZgdRv9elFwNxSn3JJOTao45tc1tSgt1Xe7bE', '341_홀핸드'], ['342', '1vhHYcd1bax1ioWgs1Z7IjxMNkwgHvPoPSR0xbxBVGDY', '342_현이네만물상'], ['344', '1ohjUVbgFjMImYFcjyuG52IFGpC1L2zJmA4FgWbrUG54', '케이세일즈(ksales)'], ['345', '1s7cgtE9Ei_HIrSD_Io5JyAAUWV5WJr6KwPAYuqalN6A', '강동성'], ['346', '13RsUAE_gHllI9mMx5X4uo4iY1RJm4Z__-0v4MmEMpkc', '삼육오푸릇'], ['347', '1ug5Xb1vodkw86jm5ZSGNx06CTljISBXtbXSwF-euHTg', '테라소셜리 네트워크'], ['348', '1JthKyD9b9lMdFUAm-AEW8v7-AFEIc3ekak-5hjGhrtM', '형제직판'], ['349', '1WqF0xezvhkCro5f7aIn3yhmmD8x5fve4sFC18qa8yn4', '해피상회'], ['350', '19ncCK2cIQG4Yu3j4AjhzHbyRHadPnU5LBeJ1Pu-q9wg', '소담유통'], ['351', '19s0dh_SDQYW0bTKfJDBfCUoOBXRNFlOKd3FDp4WwI-A', '김기준'], ['352', '1JwiXEI3pftt6jI51z5YqgFvZ3PL6waWx2tjBMm12Vzw', '잡골락'], ['354', '19pM3Ng-lhCugezu8pD1XtpmkNjlIRDu3iCn5ow3jod0', '더집'], ['355', '1t0RE5GSnCD0qaVFq9Qy99OMIXpAXimWZKa5OcKx5bnM', '김성은'], ['356', '129GLNuRCMYFUq2rFnIg0tevnLNunE-laPNmzfAyrso0', '주식회사 에이치엠씨'], ['357', '1WF_3HEscDtRpK4SzE7DEcwQg-rM3RcuKh56vEKamIHU', '헤이다니'], ['358', '16RvzfCjd8o_Pl6to47Dqe6jceLa4UsNe7YqXP985bbY', '모도리청년들'], ['359', '1xWMbewWXcVADed4O4TwMlrE5BHmaMyrBjMCOFlCCJ7o', '이효미'], ['360', '1RWo_xWOvUTHoWNOcRwd0ZQkBtnC-c6fYKwjYf9Q7txY', '와이제이'], ['361', '1ph6nZvwtOwtSD9vNSOXT4b6U_SEpHSf0vmE2IbKMmf4', '아브라함홀딩스'], ['362', '1l4VEeORrD9cbTjWvQkjksl2tbcjuREMbYFVOaDxJL2w', '조재현'], ['363', '1nwJYh9EuxPCj2bAkyxT1Cl7iSCSzgnZgWQqpoC7l0jE', '363두두푸드클럽'], ['364', '1eLWnX-nvnRDNIo8Lx3Ni4dd5bJhUL1C2Z5n9dTxkz5c', '유니네과일가게'], ['365', '124KnkRcJdxge_UPJyCOPLndfSfB82FdV0x6-fHlB1lM', '주)빈블푸드컴퍼니'], ['366', '1rssQrHgWd5oNyA33VhAst93RpWfQhyWN2SEFSMG2zIk', 'jnlhub'], ['368', '1btlBHD1o3XJ4AM1HP0Bai52UoDlLSuCb4nbqIsJz3PA', '모든농부'], ['369', '15OcobmLVmRs78bV7lOCRb6zOvF5X8meIeumf7okdoqw', '369_모든농부'], ['370', '1nHxiIwx3tuLO8IEuM3xFQIGiJ9FgAWosoJNeSquwoRo', '370'], ['371', '1MHOlWEkLAZ_ymyLvfFN9o2mnJNjZsXKAT4vYj-MSWMw', '담다컴퍼니'], ['372', '142HXZ5K0HX-hqH7zL6spzlhOhHTppUTWmrH13mCae3c', '유광선'], ['373', '1mz83gu-M3ysziEsiolCKP-YOb3PoJqRgeKYNQ0nefSM', '우정임'], ['374', '1iIqa3wcN15cSWprQyu8PHOgqZGG3kUZBYrsXBhRsFzI', '양홍석'], ['375', '1gB5yWP_BwZx2emlEt-uCwBy-09Iy8PuD372LVa-wOYE', '김진정'], ['376', '117eJeruUxOOYMQBziOtzUqM3D1MRHewNqhOQIpGTJaE', '이강무'], ['377', '1uh0OHLIf46vqaWNb-eG3A8G_KwfWVbKY0hEWDG7AgAg', '박준원'], ['378', '1bCDIco1mCPr9rRhyTBZ4Gb2siPDhAK0o-Nt9Ecb718k', '378'], ['379', '1uKeM-aENd1hq8qYe0kaOJhm1OxHeXzqmRTPLny7C4-U', '송지훈'], ['380', '1XGimHhzf-sXa7mBq2BcLL0vBpm6mSMN12vqx3731PrY', '스트롱패밀리'], ['381', '1jMissT7hmBIjRKOszZ9VsI2ctWcnDEDI9MjQGDBUIRc', '8959쇼핑'], ['382', '1zRw0pE1KgfrofHebybgrLYYigGaxjsh-STodhgX-8d0', '정하나'], ['383', '1HmdxIzxMD6Y6IAnwqNU7ywA9fZR5bBKqEp8Ku33v9g0', '박나경'], ['384', '1-R1p5imMNFRrCAi8CyoS_3KZzzmASp4vDD5ThV-xXhw', '김현진'], ['385', '1-KtcjbNPDWANWM7A7Fc6lv1wYqpeClH_NRBO9DPZ6FE', '프루티오'], ['386', '1rlqOD9wBJH6OLA8XST13fY5oD7RFATxAm7SheNXbtPo', '코끼리상회'], ['388', '1x40k6VNkyvf6t_Hobh45bZ2NLz1aKWQ-Gi5oiP4Va94', '헬로우농수산'], ['389', '12qNKS1Vuzy6pAzqAmdjkbKplQ2B0WnKkccWHxWcwjZA', '헬로우농수산'], ['390', '1d2GXQNCWAKR72FInvpTfOXXIpLxs6xcSVfmJXzNTNao', '조종현'], ['391', '1JuRJPUx3Ok79OtXgwcXn_qPzMiKMACRuYUfqFHHWH7E', '정의찬'], ['392', '1_Jg2ySHi-AWKVvqfRjzzemYPYsR0VwaERZmEQUefKio', '유앤아이'], ['393', '1MLhY1aLLkepo50gMWXYhoimyyZRg3bl6ZtxTbRh_r7k', '393_최재혁'], ['394', '1vSGsNubbtmFfAq0sUXw8YYc0SalGRUsuz4WS3y_qgN4', '권성국'], ['395', '1Qwm5tTK6QTT1W4dzhY9zhu2zePBRIvg-IU01mg_RckE', '유헌석'], ['396', '1WaBxUtIrg-xIE0vf6af7UL3XIoMnWFOcPkx5r6mHuJI', '허태욱'], ['397', '1tiac04mUgG1rGAhXPLbNgJo65Bah6eJ5V5RrHkzaNsQ', '오기연'], ['398', '1H_I3RMV0I9_gVJTFxhhT4Dbl4T9lanrYBNKTzQoOrsQ', '이인태'], ['399', '1HEc4wLxiOGI_bLnVeilrMCTgHRg4bpz_d_tA4443MJ8', '정소담'], 
        # ['400', '1V1TWc-YtD0B5niqN2hKVlIf076Q5D9I58gGXM66qzNE', '김지호'], ['401', '10LX93604qpTvmP9i4VqFw59lYIOvDwDqh_Ax0e0tGCU', '이수영'], ['402', '1K7urxalWjJmy88UFttUmW9FJ_1tGHMTbou7qNmiWoXs', '이수영'], ['403', '1TlvpZK7IfPJIrE3_xMJI7WJfrpI5rdh1AXAslLiyg_k', '문승수'], ['404', '1QApmaEqM8f8L9pTarXG5o5kWmLjiouWOvWXYviHDWi8', '서주원'], ['405', '1be90mBGwLw1Qoyt1SekMSc4_kaIdNEq4GHYcVMiyf3w', '405대한농수산'], ['407', '16Enb8YKihLDGX71hvbBX8KtJ7H-Mco-WQukPzW101zo', '윤경'], ['408', '1wgnpQR7TPJmMPh9RUfmrPiRrcOWKgqtFdkPlqFQg5fM', '장병식'], ['409', '1KcK5g3IyPTrE3LfiG4MDROCGevrq7u23R-LZLaF34No', '409'], ['410', '1vrB00dzNDDXiMrPNt93fU9NZRUcbhEWRUSSih6ZCgzs', '정은아'], ['411', '14NcftN5VzDLrpkAsNKUqErfYxWtE3fyo-qUKC3Pocak', '411남진주'], ['412', '1aVnVQJGtATIHJlW-MIL-w3VhnaJFzwImlX9ostyHoRA', '김수연'], ['413', '1ZQSrieMm8LzpQUJ_nXceEkFQSG1fntbH-6tz3ofzyUE', '박영미'], ['414', '1hLpxyUACaR5NsUxnz0eo_pcsb1FKnC4C6Mi9Dor5yWc', '조성민'], ['415', '18eG80C0I8FwKRhjUIkcK-pBDjavYARuccBRs8NyFvmM', '조성민'], ['416', '18UHw7Akk4fQMki9I5NRCMYlt9O06BUojPJihqd8-qF8', '김성유'], ['417', '1lurRKh1zUYDnaNo294VVO5Y7yPfJNevxWOqN1KnP_eg', '유재윤'], ['418', '1Mw2ilGo8wR9E0jbiwX10AUrdz3G8ImvelKTGlZEAleY', '김용욱'], ['419', '1k2ZY3LjBQL5XYnoZXU5G5nNNfnK4kjGyOsuNHUryBZs', '이상득'], ['420', '18waKEPfqhAZf1zZnOjFYO6jMXZARkb_XYfkjVs8mtig', '김문주'], ['421', '1mefN0U0TazA7nuAObVR8Jdimq1VxviqgEMErfgaLSNk', '송은지'], ['422', '1ARikzuXNMVAZpw68b8SHNEgrAx6MqgrP84fGqiDXNT0', '422스위트팜'], ['423', '1f5QXgVQn04XS-Az_jW9iaoO9-mxCXQ24KF8iTJM1WOw', '423'], ['424', '1sZ3CITq0EZUHi3brSEpXHvvlqLUm-ol5pgWMxVjx9YU', '곽재봉'], ['425', '1wqcGocCoSRhB15dmXabi272C8LMyA0cOLhVx9VDjj1g', '유수인'], ['426', '1CM6QX3m5_8IBPO3gfJ7cbXTI2uDenEQ3YSwvTKvUQhM', '김대건'], ['427', '1JZ_TWnBl08I1PqggMeMUWK4s02k0uPmsNots3aiExyM', '김유원'], ['428', '1OMZyyPFGIvm9PoGdjEZaQVsD7QvrVVNYrSWsxyPLa70', '전희원'], ['429', '1WJUVR4zy5hXE73VGCg7eqYN7CXUbZT8n35bfD_huhIs', '강재혁'], ['430', '1H-GYjjHYe1MT9Bs8ugE1lqZbzI3syFuaLdieG23GkIc', '문송현'], ['431', '1PUoixiloVPk0Gr0Dk40opzEOPTQsJ44EGKKhugz5vUM', '김영일'], ['432', '1Siz6JTNnJ9X8orvFMfkLHPl1BJEDXgetAkN3-0BOMKA', '(주)피티인사이트'], ['433', '1zJcm4Dp6_Jbt1_HPppp1jNabeLhN4VN8_xRZWYpqWSM', '김현지'], ['434', '1mwjm012LSuBAFXtEKR-l9SjZVhPS51xm1mTFQCtRL0c', '조상현'], ['435', '1plFsaj9l-2p-GNJC2XvzbS6y4AZlVMFhlNiWlvLPZmQ', '김양중'], ['437', '1FAX4xi03t15NOrrPWgx5QoiRwjgDkg-fmgs6-UDyYfg', '김홍곤(주 윤)'], ['439', '1Pt3guWLAggJ3RmZhhBW5XYdDT8Exry7acptCwJAXNdk', '오경철'], ['440', '1N9AnJdEUwtnuo4YC54OPJ8llYH3214bnAzrwKgplyTY', '정해연'], ['441', '1OQgizUzoRSNpzjUA6qvnYvyIMyadVQsTYfFxzT4cfrE', '정해연'], ['444', '1cGSDZu2qK_Cx6jXa_qzlnX1fumXlyIUi2LIKVDX70Sw', '이승민'], ['445', '1W52kNbCaDXafLTYxMfJ3pnMmzU8o1UIkYonsrmm2V1k', '정영식'], ['446', '1FAInvjJCqdCk1ZyoFWv-kUhF6pBmZpN43AK_z_Exkjs', '박서희'], ['447', '1CwPGQ9T353ee1YZRy3HVB1KnF2ay0cZkA5LLUXpXrrU', '김서온'], ['448', '1_LMtNqEmzXV9Fm_3X7N6C5xgTjKNsHrH8tO_4zyK7yU', '이상원'], ['449', '1ug6Cq86Uyqd1GaUMp-SLiLhR9Bl37QIU0Dp-JvXVFMU', '449_샘스랩'], ['450', '1zjp2aZRe5kOnnV1V5zpuEq9KASE3zSpmKr9_hLBl-MA', '이서율'], ['451', '1hVWqPBOI7pvoYteC2ts7XeKisIfTEDIbRIPtbOYA9MQ', '이서율'], ['452', '1v0MX1YZ-BMTKdylF8reNRAf-9zzL5RqrERrhasZaTLQ', '최민수'], ['453', '1uXcc-VYZFV2qLwocgAlY9XTzXCP_-0gkGQe5YuUAqN8', '박수현'], ['454', '1hp4jKgpr3wvpboEaN-eiDMlTusLa1oZxf-08c8Soo40', '박수현'], ['455', '1qIK7MsVk0bB-zt7_OkfMP4Tu4xbh77hmlMdS_dLQOSA', '안태혁'], ['456', '1oOWKbvHYbE_XH5OQSSzuWxlvNaIcnlcMBq-aZbabsb0', '김민경'], ['457', '1pL_nAvqcZMgXnT3xtJu0hdqFvzmm-gobaHJSzStDY2Y', '조준호'], ['458', '11D86Ux0Bp4WlCWXwAGzESQUcHuPVHPzJV_ZKwYhc1vE', '김혜정'], ['459', '1-shYws5SdWblFyiq9IRlxNAwwFyyjb6VvDzqPFAuaDo', '백시연'], ['460', '1concNI0rwdwObLN-lnm-HAw1JGU0BgTwuLD4yMndp1Q', '모닝큐'], ['462', '1g1Wt26k_G9i6gx3j2B-ogZ7O2VVeJRv6OjXjKmPQmfc', '김찬진'], ['463', '1LjeX4IP8i13tKX_HqQgCvxM3k45QpdGluQtnQiOOaOQ', '최수지'], ['465', '1P6p_fSZ-E28ThAwYOzanRXHKDuyPyVqLDTtyexlI3EY', '최정임'], ['466', '1qCEZqPwP8x5FNOU038iSwx4M_cP_uP9J1Yhuv7qFk-0', '나승은'], ['467', '1N8OZ2HkeX7woyNezM9OP4hQtZmhhj7YuDhWrjW645JQ', '임하나'], ['469', '1l_dyacnX7lhxtPot1czzBiWEnLODcVuEJ77vVddfDdw', '조윤희'], ['471', '1HOqghnEAY675vqMMY2wM2U_N5gvM24zF0yXk6VaKugs', '조윤희'], ['472', '1yfWE9wWcg_GU_i6wXmqIWN1sZHfc6CSh5KmaJ-_olZU', '한흥구'], ['473', '1fXFH1Y0KQZuymcYaASF7-zwcvnuD_wW4yZnLz7eLxv4', '노민석'], ['476', '1XO6LrLWncVDmQwUe9PYpi3I5nZXasqEIkuD3p_OMlYI', '최형우'], ['477', '1HKSElhBwf_3PolkLJXv9ObTRBOlfwt3DTHrnNgMbmhk', '김남식'], ['478', '1gRbY6nv8qD6DqSNoGVonBqQGf9hQGauWYZEB0HHRBuQ', '서은지'], ['479', '1YRCBbhi4Rk7rRXK5k-EOLZVNjRjCf2uO9lVyV3qcE7k', '최나리'], ['480', '1R_woSV7j9Fr1PKJ0vJ81f4Jf_L9LeyHOZsanXZdR3g4', '강대혁'], ['481', '1Qiojz3YWrfAZbxaD7aNwYSQayzrOT58_dAJ46p-EapI', '김송화'], ['482', '1isujZYzUS2pT-nuUrRE9P6pMAAQeeo_sSnyK2_69exQ', '이선애'], ['483', '1IazZfVVMAC5EWwiMTb0LQXsEzw2Hvq0RAzyJ518iZsk', '김현규'], ['484', '1D1g5pQyTcdH5JTyj8C23On7nL6XTSUaIUkbpMmrcqj8', '김태우'], ['487', '1Dmg4tkV3YsN4dMJmNEL4-BF0G4kGYy6xllueuozHw-A', '오경자'], ['488', '1tnUWsjNd_oj7_LnDAgYbXC9Ev29_On5X1KbidqK-fqM', '유지연'], ['490', '1brzKZV9C0o6NQ-lWbOF1l0KIelzzwhwxUVB4qcmPJcE', '이진우'], ['491', '1Ik_Mh0dpp1jMAYNBMj_k-2pTbVz3R4lHOQfQHINhq08', '윤혜진'], ['492', '1WeXxJ9UVmuZ2CxnqG36dkcp8UFK48Ucbf1rxytCq0r0', '유영훈'], ['493', '1Yv3r3WYFfYtvWE9xqgUfcxMvMF7shM88lR64BsX3MXI', '오늘일과'], ['494', '1iiW7q0Giw2TGM3CURbI3fFmku7ZwFePnRfK0s9UPgmw', '494_김소영'], ['495', '1hDEfrKjerlTNmvt9vTb37CJKQgegcWwP3Y0W3-nLPGQ', '이지용'], ['499', '1IVcgWv_hGgcjmQhYNsJ2j7DRi6sv7PY5pZZAn_mcHfY', '이지용'], 
        # ['501', '1f7aa_nP4scVscIVqL0aRVho-fk6U-ftwA23fliYDUnU', '장하당김도하'], ['502', '1cevtTyAdU9HZYt0O0FVArzifTSTO0IVdZVYohC1NMOw', '김성헌'], ['505', '10s_LJ7-VUq8HabuAeXQIaRpC7e2TpF0RV02j7JPAMWQ', '505모정훈'], ['507', '1tgPO2nT2rkKVf90XdKbcAvak3PAirTqp4Hib7JJqWhM', '류진경'], ['508', '1zvh9CT8ae9XqXA3zoR4XLbOFRFWDCXSrrSTQUNWm_0k', '박정숙'], ['509', '1EKkvG2AnR1p5oahgiP8qHs2pL1IdSOTZG0O4DYjnYt0', '박현희'], ['510', '1URrcE4La06zIoEvCbSZcrpo3AngfU_EiO5uZ8WdmiAY', '조혁준'], ['511', '12l-0fx_ygkUwkPbzZSb6KvEobUabyU1HodDBftdfdTc', '최현주'], ['512', '1PHpg2H2W-ChCOsJyIOebqf7LfcryrOBgHG5D6AYFIHI', '512'], ['515', '1-0alpGTk3TATHYuDJn3-vtlTpJ40hRrQYY0aINWUjLk', '전성훈'], ['518', '1Ycvhq1jXkg9XDTtGoXiFoPRggMLHjCXQlFxlxD__LPM', '박민지'], ['519', '1DQ9qzOMVa5ic14wbic6QUQBcLivqxQ4R7Q5Fmu2hUMc', '박진원'], ['520', '1NJdWWovSrDQFWEPh3Q3G_4RTbjQ6myVpzAdgsUlR7J0', '윤진'], ['521', '1s_SUz4NYsghHi66D2KI6tMpBXnytcyovPGY8lKeLJuM', '토스 홍주연 ((비바리퍼블리카（토스）-오픈뱅킹))'], ['522', '1-VebAqmGFVptv5180nn6axOuS-1iTSj642cdME-PwmA', '안찬규'], ['524', '10rwhbPk7fAEszGkalL2e21BxW1lU5Ee1DRra7i1di4o', '마기영'], ['526', '1Wy4S5QWmq5QsZcqfFRT29bMBoB9Y6x3vPtCI2jV6rek', '네이처파머'], ['528', '1kscIkrMDFGD-f_tdkS7AlBsAmfzJ1zM3vbIAS3z0lh8', '박진헌'], ['529', '1bMeMOKIpR81NOpcVlpX1YMD8ZDIHTgzE3QxDzjFoXkc', '장우영'], ['530', '1mxpwoEGSws1oNeBQJego68yGYgog2EGCfs3-2T6ItgE', '디에스코퍼레이션'], ['531', '1sS5YFIFodb2xBQqzcbUQYra5zZgY5pUD3-cRoeFpV60', '박정아'], ['532', '17TmNLanvPGtrl7-2zuPSjVVPlySgGJWwKL6nYYPE5tU', '532_박정아'], ['533', '15kjs_sGHIeO-lEXsRtke9w279aD2Gly73r6Gwmdv3iw', '김혜연(온앤온맘스토어)'], ['534', '1fJnXopkC9y9c4J_wLlAYAn_DmUajLCIT0yLMcveZR_c', '박소진'], ['535', '14cfV8zU1OJnTsujjLKM_D5gCuytyr_CV0odStQDUvWw', '조은정'], ['536', '14O6xioG8YSjP0VTaV2kLNEHe38lXvEbpmyLlRI2jwPQ', '536_조은정'], ['539', '1oAiMzvScQRhec0v_27ZDxzShsSndVnL8SNhhdvwCZ1A', '이주희(키즌)'], ['540', '1JIIiXZkPMGJSs4d7rLLOHUEz510tC1-gl5Cbo1YyzyE', '박수진(데이위드유)'], ['544', '12w9iiNxR5wtuWFOUQm7rc0uJtL98HoEaMKeVCw9TG7Y', '한지은'], ['548', '1gwsM3JyCn2hnYJeAQKj2Awfpb-7e5MdEhHjpH7bNlBY', '디자인위즈인(임현선)'], ['549', '1uYVa9I_wlB5RpbcRFQ03e49fO-5nmsA569cD6p1bD8g', '한지은'], ['550', '1b2vo5VoQdzlO-dUlTcp6MQknqRRuP1X5QhcOUJt2F6E', '550_한지은'], ['551', '17_thLqihNpg-f09mWZkb5SagPoqOVAPDGM4rvLQ5T9s', '채유신'], ['556', '1jtGYS6MBtrqgZMpkZnFafPyNTVGDxu_qOcd5STgi2vY', '김병찬'], 
        # ['557', '1RKDJjPRYLmMQbRKwwLCMHA3uUQB3I5VDVjqsle2ocIM', '박지산'], ['558', '10dC0yLuRSpCWlydxBR1yty6HkrvLzz_99iU_F4HbyAE', '오재원(구와일과채소)'], ['560', '1NSwZqJdKU46moIUlKrXEjP8FpINL7fjOyxk4ZXJTReM', '최재혁'], ['561', '1yLURni5N03TeMxF4RugKa9UAxJ6VyTstGztM2byv9sY', '채유신'], ['563', '1zNtcuVzLg4wn60AbDDkPL13AgUAaRV1JP2w7bvxidYM', '최가영'], ['564', '1inuqBQ4PLW5mKWqjBmYn1wRfhFsc6ngQqZPWr2obHVY', '홍영국'], ['565', '111fmUnh9i7hBagFAFXH2dD-74VCfYeh2dOenY-TvYjM', '신혜영'], ['566', '1a4p9c1IHbJebxY-i8veDrqlF1BTg9ymJleOuxWHHZZI', '최종상'], ['567', '1FozmCoU7iVClBNUNtaGHIW4clKXgId8D2DtpXy9lht0', '살림혜윰'], ['568', '1bGMJcml3PyU2xZYSgbJayRUvl6WLYxwutGEPV6ACBvg', '남지인터내셔날'], ['569', '1d7y1ce0UFiD0iJLcGf1Ro7Q1-luu-cm1J0dWaUmn-iA', '행호상회'], ['570', '1dtFP9V3cYnuZI0dmFMrDXjKOLI1ksmnPRdaTM9itNKw', '주니유통'], ['571', '1E-P_pweQgTKW4lJSqMvUrKLiimsxuHZI7qP1w0PjgOg', '571_김예람'], ['572', '1EiOg2zgHq7aLkMae9Js-RX_JQPTcULWV8mcAsNY7-A0', '572_박성웅'], ['573', '1tNi8pEL_N5BGCCXYKnTywkutg9xwXYnpMvsuOOnNfqc', '박태준'], ['574', '1uEfQ_QpcKkSnBCSeP2MzF-LOK_sTTtLE2LGnFvgENaI', '오경자'], ['575', '1dbtwaOOVdCCx7n4fE7HEIajmQaeQDAKbFtLQkuReO8A', '이환배'], ['576', '1OCC9qOF9RUlMrzkjxAb6hc9W5xpSXt5qoexmNGlqt-0', '(주)다솜코퍼레이션'], ['577', '1CCbyIQG6gSNsQTguOwefIwOzlrAgrsDL99XTsQjhtl0', '최가영'], ['578', '1cxY5XRMIwrDw1proZx0_WJ6pO3AtW0GO1xyzT3zg8wM', '578_최가영'], ['579', '1N-hCmoFuVZ7dBLk5DmJ8xUR9HfImWRYVLrCxkdQNie0', '우리가족농산물'], ['580', '1uN9pvPwnDlMOkUzbZA3-HFJqCAyggsqljQ6HLc6SAsg', '580_김두예'], ['581', '16phngvRW3i1tp67Go2BdnOLW6OsnVPrNx029G-jRxWg', '581_장준혁'], ['582', '1DfcnRgtSb3ys6zF9ewgK3_900e01uiYsG-IKdKx_lNU', '조현식'], ['583', '1LxHZy8DLQT4mc_JBOheC2N2UoNZ-dV73WQUz8dCX8hA', '583_조현식'], ['584', '1nocFQA-DyXdNVTCWKKhZ8oHc3aW5wYfhQbF-vcB8Bp8', '584'], ['585', '1T6-0dJ5ddQqSYdZmliU056Oe5Xu1n6tJfNocFr9pdbc', '585'], ['586', '1VS1KKfK7pE48_uB5TT511QUl954eL9_cfR1KuOjqb9k', '586'], ['587', '1uwb8uRFJjdLU2RhkpCpjuQsr9e_5Q86DLJ0JErJZ5Io', '587'], ['588', '1Jv6YOyPg1LdszQvEkC-gsZt77zlN-JKnWfFED3Sc4M4', '588'], ['589', '1k4L4s1uctvjYjfoFA-ROusUg7DbOdzb8gWgjUPXEiyQ', '589']
        ['19', '1yvjzHlr3A5M8wIa1WZZ7I9LlLEcmZwtniwip_ZU1w8s', '김태형(무브스몰'], 
    ]
    
#     filtered_user_data = [
# # [581,	"1YwT7BaFpOKB2Ng3MViX7UGuF9hZJUJRBFJN3eXn4gHw"]
# # [581, "1-vrWkYDHqDmhtkV5ZylS_AvmbKjrfAncu1atMKQQ8f0"]
# # [461, "104ZWc3RoB0WjkITYQLyFBLvkZ5UywvIJZHoWIha9iTk"]
# # ["516",	"1VhHatV-asswfTDa9M13B8Orf2aDSmbdYWBzwrI4dqjM", "485정준희"]
# # ["485",	"1vVGjqAgaOFovhHhrcpGkHmGktJug0f0zKgYIERT2FTA", "485정준희"] 싸우고있어서 아직못함
# # ["442",	"1E2i1IJRuPAoDmXum5vjrNo65mmaEjIyBKhENEH7VZEE", "임경섭(프루딧)"],
# # ["555",	"1Pu9P4zFoDdFXK9ER_rZaEj2fa0FX36YEyCZIAV-YkEw", "(주)마니쿡스"],
# # ["523",	"1y6sJA3EAAqaSLPgZMWYJjY5t2IZLrZGMBTXWI7sroIg", "김지영"],
# # ["339",	"1GCUBnyUr25uhB3HozLbiUQ5JngLhr7rfoNNR1cxAo58", "339슈거팜"]
# ["504",	"1YwT7BaFpOKB2Ng3MViX7UGuF9hZJUJRBFJN3eXn4gHw", "504_팜브릿지"],
# # ,["325",	"14SbtLIs4ZPfiaJh-kRb-TE4RtOcO-gw4knuy3MolLX8", "이호연(컨프래"]
# # ,["114",	"1mZVlL4QwI2eHByQEpYs0dLYUQX4OcSa4tsdTJPR2VRI", "114"]
#     ]
    # result = [["560", "1jhYbEM98arK-Xj5cSQ01P2Oa3kfgO6lNODw6tDqEFPU"], ["561", "1yLURni5N03TeMxF4RugKa9UAxJ6VyTstGztM2byv9sY"], ["562", "1jYeZFz3CNgXkBV3z1CsoD8k_e6xH-vZa3M0KboBD8rk"]]
    # print(filtered_user_data)
    # importrangeAllowAccessAutomation("1YwT7BaFpOKB2Ng3MViX7UGuF9hZJUJRBFJN3eXn4gHw", product_spreadsheet_id, creds)
    # raise Exception("is in the excluded list. Stopping execution.")
    
    excluded_titles = [21, 335, 442, 504, 516, 523, 555]
    for data in filtered_user_data:
        print("data")
        print(data)
        spreadsheet_title = data[0]
        deposit_spreadsheet_id = deposit_spreadsheet_list[-1]
        list_spreadsheet_id = product_spreadsheet_list[-1]
        # if int(spreadsheet_title) in excluded_titles:
        #     continue
        
        # if 19 > int(spreadsheet_title):
        #     continue

        if 610 >= int(spreadsheet_title) > 511:
            deposit_spreadsheet_id = deposit_spreadsheet_list[-1]
            list_spreadsheet_id = product_spreadsheet_list[-1]
        elif 510 >= int(spreadsheet_title) > 411:
            deposit_spreadsheet_id = deposit_spreadsheet_list[4]
            list_spreadsheet_id = product_spreadsheet_list[4]
        elif 410 >= int(spreadsheet_title) > 311:
            deposit_spreadsheet_id = deposit_spreadsheet_list[3]
            list_spreadsheet_id = product_spreadsheet_list[3]
        elif 310 >= int(spreadsheet_title) > 211:
            deposit_spreadsheet_id = deposit_spreadsheet_list[2]
            list_spreadsheet_id = product_spreadsheet_list[2]
        elif 210 >= int(spreadsheet_title) > 111:
            deposit_spreadsheet_id = deposit_spreadsheet_list[1]
            list_spreadsheet_id = product_spreadsheet_list[1]
        elif 111 >= int(spreadsheet_title):
            deposit_spreadsheet_id = deposit_spreadsheet_list[0]
            list_spreadsheet_id = product_spreadsheet_list[0]



        destination_spreadsheet_id = data[1]
        print("spreadsheet_title", spreadsheet_title)
        depositer_name = ""
        if len(data) > 2:
            depositer_name = data[2]



        # if int(spreadsheet_title) < 580:
        #     print("580이하입니다")
        #     break
        await main(service_sheets, destination_spreadsheet_id, source_spreadsheet_id, spreadsheet_title, depositer_name, deposit_spreadsheet_id, list_spreadsheet_id)
        
       

        time.sleep(0.5)

        print("Calling importrangeAllowAccessAutomation for archive_spreadsheet_id")
        try:
            importrangeAllowAccessAutomation(destination_spreadsheet_id, deposit_spreadsheet_id, creds)
            print("importrangeAllowAccessAutomation for archive_spreadsheet_id executed successfully")
        except Exception as e:
            print(f"Error during importrangeAllowAccessAutomation for archive_spreadsheet_id: {e}")


        print("Calling importrangeAllowAccessAutomation for product_spreadsheet_id")
        try:
            importrangeAllowAccessAutomation(destination_spreadsheet_id, list_spreadsheet_id, creds)
            print("importrangeAllowAccessAutomation for product_spreadsheet_id executed successfully")
        except Exception as e:
            print(f"Error during importrangeAllowAccessAutomation for product_spreadsheet_id: {e}")
        

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=520657704422-o724cfbjdm5vnt6iko73r4buo8gjr68k.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fspreadsheets+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fscript.projects+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.file&state=tUZWQ5YYF6zdHDNPBN0W4SZSFHspkr&access_type=offline
data
['19', '1yvjzHlr3A5M8wIa1WZZ7I9LlLEcmZwtniwip_ZU1w8s', '김태형(무브스몰']
spreadsheet_title 19
Calling importrangeAllowAccessAutomation for archive_spreadsheet_id
JSON 디코딩 실패 )]}'
{"success":true}
importrangeAllowAccessAutomation for archive_spreadsheet_id executed successfully
Calling importrangeAllowAccessAutomation for product_spreadsheet_id
JSON 디코딩 실패 )]}'
{"success":true}
importrangeAllowAccessAutomation for product_spreadsheet_id executed successfully


In [25]:
import json

creds = oauthByWeb()


def test(get_spreadsheet_id, give_spreadsheet_id, creds):
    url = f'https://docs.google.com/spreadsheets/d/{get_spreadsheet_id}/externaldata/addimportrangepermissions?donorDocId={give_spreadsheet_id}&token={creds.token}&includes_info_params=true&cros_files=false'
    # https://docs.google.com/spreadsheets/d/1YwT7BaFpOKB2Ng3MViX7UGuF9hZJUJRBFJN3eXn4gHw/externaldata/addimportrangepermissions?donorDocId=1Fg76vk3lucUwwvmcoQJVb4KiMfjJSNx2VEFSdhwYFy4&token=AC4w5VjkIUgDXtWZH968yYaAfiyhZJcApg%3A1718330258244&includes_info_params=true&cros_files=false

    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {creds.token}',
    }
    # 데이터를 JSON 형태로 변환하고, POST 요청을 보냅니다.
    response = requests.post(url, headers=headers)
    # 응답 확인
    if 'application/json' in response.headers.get('Content-Type', ''):
        try:
            print(response)
            # 응답 상태 코드 확인
            if response.status_code == 200:
                # JSON 응답 파싱
                data = response.json()
                # 파싱된 데이터 출력
                print(data)
                print(response)
            # data = response.json()
            # print(data)
        except ValueError:
            print("JSON 디코딩 실패", response.text)
    else:
        print("응답이 JSON 형식이 아닙니다:", response.text)

# test("1YwT7BaFpOKB2Ng3MViX7UGuF9hZJUJRBFJN3eXn4gHw", "1XDBIK9o2k4AVMcW2gfieDZ4qSBxcTW8npw596AY6KCY", creds)
# importrangeAllowAccessAutomation("1VhHatV-asswfTDa9M13B8Orf2aDSmbdYWBzwrI4dqjM", "1vXx5vFkPWRsxE5OgfaB4t2beO6QFlBy3qUwFjRb6gtY", creds)

In [26]:

# if __name__ == "__main__":
#     spreadsheet_id = "1YwT7BaFpOKB2Ng3MViX7UGuF9hZJUJRBFJN3eXn4gHw"
#     creds = oauthByWeb()
#     service_sheets = build('sheets', 'v4', credentials=creds)

#     sheet_id_list = [2108815336, 1616742714, 2091182696, 66701082, 738982729]
#     ordersheet_number_title = "504"
#     depositer_name = "504_팜브릿지"

#     sheet_order_id_발주서 = sheet_id_list[0]
#     sheet_order_id_누적발주 = sheet_id_list[1]
#     sheet_order_id_입금내역 = sheet_id_list[2]
#     sheet_order_id_상품목록 = sheet_id_list[3]
#     sheet_order_id_매핑모드 = sheet_id_list[4]

#     # request_name_index = [
#     #     apply_rename_request(sheet_order_id_발주서, "발주서"),
#     #     apply_rename_request(sheet_order_id_누적발주, "누적발주"),
#     #     apply_rename_request(sheet_order_id_입금내역, "입금내역"),
#     #     apply_rename_request(sheet_order_id_상품목록, "상품목록"),
#     #     apply_rename_request(sheet_order_id_매핑모드, "발주서매핑모드"),
#     #     apply_index_request(sheet_order_id_발주서, 0),
#     #     apply_index_request(sheet_order_id_누적발주, 1),
#     #     apply_index_request(sheet_order_id_상품목록, 2),
#     #     apply_index_request(sheet_order_id_입금내역, 3),
#     #     apply_index_request(sheet_order_id_매핑모드, 4)
#     # ]
#     requests = []

#     ordersheet_number = int(ordersheet_number_title)

#     result_입금자명_발주서 = apply_formula_to_text(str(depositer_name), "J4:J4", sheet_order_id_발주서)

#     result_발주서번호_발주서 = apply_formula_to_text(str(ordersheet_number_title), "T4:T4", sheet_order_id_발주서)

#     formula_예치금잔액_발주서 = f'=IMPORTRANGE("https://docs.google.com/spreadsheets/d/1tRaty5BdhhVWrBvsx-SvMBHnnJaZZRekh2-4SJnFuqc/edit#gid=516836762", "\'셀러발주서정보\'!D{ordersheet_number + 1}:D{ordersheet_number + 1}")'
#     result_예치금잔액_발주서 = apply_formula_to_function(formula_예치금잔액_발주서, "K4:K4", sheet_order_id_발주서)

#     formula_입금확인중_발주서 = f"=IFERROR(SUMIF(\'누적발주\'!H4:H, \"입금확인중\", \'누적발주\'!R4:R), 0)"
#     result_입금확인중_발주서 = apply_formula_to_function(formula_입금확인중_발주서, "O4:O4", sheet_order_id_발주서)
#     formula_추가입금필요_발주서 = f"=IFERROR(SUMIF(\'누적발주\'!H4:H, \"입금확인필요\", \'누적발주\'!R4:R), 0)"
#     result_추가입금필요_발주서 = apply_formula_to_function(formula_추가입금필요_발주서, "P4:P4", sheet_order_id_발주서)


#     formula_예치금잔액_발주서매핑모드 = f'=IMPORTRANGE("https://docs.google.com/spreadsheets/d/1tRaty5BdhhVWrBvsx-SvMBHnnJaZZRekh2-4SJnFuqc/edit#gid=516836762", "\'셀러발주서정보\'!D{ordersheet_number + 1}:D{ordersheet_number + 1}")'
#     result_예치금잔액_발주서매핑모드 = apply_formula_to_function(formula_예치금잔액_발주서매핑모드, "K4:K4", sheet_order_id_매핑모드)

#     formula_입금확인중_발주서매핑모드 = f"=IFERROR(SUMIF(\'누적발주\'!H4:H, \"입금확인중\", \'누적발주\'!R4:R), 0)"
#     result_입금확인중_발주서매핑모드 = apply_formula_to_function(formula_입금확인중_발주서매핑모드, "O4:O4", sheet_order_id_매핑모드)
#     formula_추가입금필요_발주서매핑모드 = f"=IFERROR(SUMIF(\'누적발주\'!H4:H, \"입금확인필요\", \'누적발주\'!R4:R), 0)"
#     result_추가입금필요_발주서매핑모드 = apply_formula_to_function(formula_추가입금필요_발주서매핑모드, "P4:P4", sheet_order_id_매핑모드)


#     formula_입금내역 = f'=IFERROR(QUERY(IMPORTRANGE("https://docs.google.com/spreadsheets/d/1tRaty5BdhhVWrBvsx-SvMBHnnJaZZRekh2-4SJnFuqc/edit#gid=322712525", "전체입금!A2:E"), "SELECT Col1, Col2, Col3, Col5 WHERE Col5 = {ordersheet_number_title} ORDER BY Col3 DESC"), "")'
#     result_입금내역_입금내역 = apply_formula_to_function(formula_입금내역, "A2:A2", sheet_order_id_입금내역)

#     formula_상품목록 = f'=IMPORTRANGE("1Fg76vk3lucUwwvmcoQJVb4KiMfjJSNx2VEFSdhwYFy4","\'시트1\'!A2:M930")'
#     result_상품목록_상품목록= apply_formula_to_function(formula_상품목록, "AA1:AA1", sheet_order_id_상품목록)
#     result_코드번호_발주서 = apply_formula_to_function_in_everyrow(sheet_order_id_발주서, ordersheet_number_title)



#     result_입금자명_발주서매핑모드 = apply_formula_to_text(str(depositer_name), "J4:J4", sheet_order_id_매핑모드)
#     result_발주서번호_발주서매핑모드 = apply_formula_to_text(str(ordersheet_number_title), "T4:T4", sheet_order_id_매핑모드)


#     result_dropdown_발주서 = apply_dropdown_validation("\'상품목록\'!$K$6:$K$800", "D12:D941", sheet_order_id_발주서)
#     result_dropdown_발주서매핑모드 = apply_dropdown_validation("\'상품목록\'!$K$6:$K$800", "D12:D941", sheet_order_id_매핑모드)
#     # print(result_dropdown_발주서)

#     requests.extend([result_dropdown_발주서])
#     requests.extend([result_dropdown_발주서매핑모드])



#     # requests.extend(result_주문상품_발주서)
#     # requests.extend(result_상품매핑확인_발주서)
#     # requests.extend(result_공급금액_발주서)
#     # requests.extend(result_발주상태_발주서)
#     # requests.extend(result_수량_발주서)
#     # requests.extend(result_코드번호_발주서)
#     print("result_dropdown_발주서")
#     print(result_dropdown_발주서)
#     print("result_dropdown_발주서매핑모드")
#     print(result_dropdown_발주서매핑모드)
#     print(requests)
#     body = {
#         'requests': requests
#     }

#     # batchUpdate 호출
#     response = service_sheets.spreadsheets().batchUpdate(
#         spreadsheetId=spreadsheet_id,
#         body=body
#     ).execute()


# # result_code_number_코드번호_발주서 = apply_formula_code_number_to_function_in_everyrow(sheet_order_id_발주서, ordersheet_number_title)

# # result_dropdown_발주서 = apply_dropdown_validation("\'상품목록\'!$K$6:$K$800", "D12:D940", sheet_order_id_발주서)

# # requests.extend(result_dropdown_발주서)

# # request_protect = [
# #     apply_protect_data(sheet_order_id_발주서, "J4:O4"),
# #     # apply_protect_data(sheet_order_id_발주서, "T1:Z940"),
# #     apply_protect_data(sheet_order_id_상품목록, "I1:BA1000"),
# #     # apply_protect_data(sheet_order_id_입금내역, "A1:Z940")
# #     ]

# # request_function_index = [
# #     result_입금자명_발주서, 
# #     result_발주서번호_발주서,
# #     result_예치금잔액_발주서, 
# #     result_입금확인중_발주서,
# #     result_추가입금필요_발주서,
    
# #     result_입금자명_발주서매핑모드, 
# #     result_발주서번호_발주서매핑모드,

# #     result_예치금잔액_발주서매핑모드,
# #     result_입금확인중_발주서매핑모드,
# #     result_추가입금필요_발주서매핑모드,


# #     result_입금내역_입금내역, result_상품목록_상품목록
# # ]   

# # requests.extend(result_코드번호_발주서)
# # requests.extend(request_function_index)
# # requests.extend(request_name_index)
# # requests.extend(request_protect)
